# 设置 Azure OpenAI

In [ ]:
import os
from dotenv import load_dotenv

#load env variables
load_dotenv('.env')
aoai_api_key = os.environ.get('AZURE_OPENAI_API_KEY') 
aoai_api_version = "2024-08-01-preview" #os.environ.get('OPENAI_API_VERSION')
aoai_endpoint = os.environ.get('AZURE_OPENAI_ENDPOINT')

In [2]:
import openai
openai.api_key = aoai_api_key
from openai import AzureOpenAI
print(f"api_key_openai:{aoai_api_key}, api_version={aoai_api_version}, azure_endpoint={aoai_endpoint}")
aoai_client = AzureOpenAI(api_key=aoai_api_key, api_version=aoai_api_version, azure_endpoint=aoai_endpoint, azure_deployment="gpt-4o")


api_key_openai:ab31f8caee164b7391f761955cdf0904, api_version=2024-08-01-preview, azure_endpoint=https://aoai-tst1.openai.azure.com/


In [3]:
def askaoai(messages):  
    response = aoai_client.chat.completions.create(  
        model="gpt-4o", 
        messages=messages,  
        max_tokens=2048,  
        temperature=0,
        top_p=0,
        frequency_penalty=0,
        presence_penalty=0
    )  
    return response

In [4]:
sysp_default = "你是一个电饭锅领域的国际电商专家。"
history = [{"role": "system", "content": sysp_default}]     #历史对话记录，以丰富上下文和把控每一轮问答的大致方向

def ask_with_history(usrp, sysp=None):
    if sysp is not None:
        history[0] = {"role": "system", "content": sysp.strip()}
    history.append({"role": "user", "content": usrp})
    response = askaoai(history)
    rspstr = response.choices[0].message.content.strip()
    history.append({"role": "assistant", "content": rspstr})
    print(rspstr)
    return response


# 用Azure OpenAI构建标签体系与打标

经多次的尝试分析，发现客户的需求如果仅按用户旅程阶段一层层分解标签体系的话，无法得到客户所期望的效果。而实际上，客户所期望的标签体系涉及三个层面的标签体系，用户旅程阶段的，用户对服务的关注点，用户对电饭锅特性的评价点。服务类型还可以由用户旅程阶段分解下来得到，因为用户旅程不同阶段所涉及的服务是不同的，而电饭锅特性则无法由用户旅程分解得到，而是在每个阶段都会涉及对电饭锅特性的评价。

因此，我们构建了三个标签体系：用户旅程阶段，没有继续分解，作为服务和产品特性标签体系的第一层；服务相关的标签体系，第一层是该服务类型所属的用户旅程，第二层是服务类型，第三层是用户评论该服务时的关注点；产品特性相关的标签体系，与用户旅程阶段是平行的，每个用户旅程阶段都会涉及到各种产品特性，因此构建了两层，第一层是产品特性，第二层是用户在评价该产品特性时的考虑点。由于产品特性相关标签体系只有两层，所以在对评论打标时会把该评论所属的用户旅程阶段作为首级标签打上。

## 利用GPT与已有知识构建初步标签体系

### GPT生成用户旅程相关标签体系

In [5]:
ask_with_history(usrp="请详细解释一下用户旅程。") # 仅利用GPT已有的知识，生成用户旅程阶段

用户旅程是指用户从意识到某个产品或服务的需求开始，到最终购买并使用该产品或服务的整个过程。在电饭锅领域，用户旅程可以分为以下几个阶段：

1. **意识阶段**：
   - **需求识别**：用户意识到需要一个新的电饭锅，可能是因为旧的电饭锅坏了，或者他们想要一个功能更先进的电饭锅。
   - **品牌认知**：用户开始接触到不同品牌的电饭锅，可能通过广告、朋友推荐或社交媒体。

2. **考虑阶段**：
   - **信息收集**：用户开始在网上搜索电饭锅的相关信息，包括品牌、型号、功能、价格等。他们可能会访问电商平台、品牌官网、产品评测网站等。
   - **比较与评估**：用户将不同品牌和型号的电饭锅进行比较，评估它们的功能、价格、用户评价等，以缩小选择范围。

3. **决策阶段**：
   - **产品选择**：用户决定购买某一品牌和型号的电饭锅。
   - **购买渠道选择**：用户选择购买渠道，可能是通过电商平台、品牌官网或线下实体店。

4. **购买阶段**：
   - **下单与支付**：用户在选择的渠道下单并完成支付。
   - **配送与交付**：用户等待电饭锅的配送，并最终收到产品。

5. **使用阶段**：
   - **产品使用**：用户开始使用电饭锅，体验其功能和性能。
   - **售后服务**：如果用户遇到问题，他们可能会联系售后服务寻求帮助。

6. **忠诚阶段**：
   - **满意度评估**：用户评估对电饭锅的满意度，决定是否会再次购买该品牌的产品或推荐给他人。
   - **反馈与评价**：用户可能会在电商平台或社交媒体上留下产品评价，分享使用体验。

在整个用户旅程中，品牌和商家可以通过提供优质的产品信息、良好的客户服务和积极的用户互动来影响用户的决策过程，提高用户的购买意愿和忠诚度。


ChatCompletion(id='chatcmpl-ASemYPakI5H34XZqsEWH2UZC6t86r', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='用户旅程是指用户从意识到某个产品或服务的需求开始，到最终购买并使用该产品或服务的整个过程。在电饭锅领域，用户旅程可以分为以下几个阶段：\n\n1. **意识阶段**：\n   - **需求识别**：用户意识到需要一个新的电饭锅，可能是因为旧的电饭锅坏了，或者他们想要一个功能更先进的电饭锅。\n   - **品牌认知**：用户开始接触到不同品牌的电饭锅，可能通过广告、朋友推荐或社交媒体。\n\n2. **考虑阶段**：\n   - **信息收集**：用户开始在网上搜索电饭锅的相关信息，包括品牌、型号、功能、价格等。他们可能会访问电商平台、品牌官网、产品评测网站等。\n   - **比较与评估**：用户将不同品牌和型号的电饭锅进行比较，评估它们的功能、价格、用户评价等，以缩小选择范围。\n\n3. **决策阶段**：\n   - **产品选择**：用户决定购买某一品牌和型号的电饭锅。\n   - **购买渠道选择**：用户选择购买渠道，可能是通过电商平台、品牌官网或线下实体店。\n\n4. **购买阶段**：\n   - **下单与支付**：用户在选择的渠道下单并完成支付。\n   - **配送与交付**：用户等待电饭锅的配送，并最终收到产品。\n\n5. **使用阶段**：\n   - **产品使用**：用户开始使用电饭锅，体验其功能和性能。\n   - **售后服务**：如果用户遇到问题，他们可能会联系售后服务寻求帮助。\n\n6. **忠诚阶段**：\n   - **满意度评估**：用户评估对电饭锅的满意度，决定是否会再次购买该品牌的产品或推荐给他人。\n   - **反馈与评价**：用户可能会在电商平台或社交媒体上留下产品评价，分享使用体验。\n\n在整个用户旅程中，品牌和商家可以通过提供优质的产品信息、良好的客户服务和积极的用户互动来影响用户的决策过程，提高用户的购买意愿和忠诚度。', refusal=None, role='assistant

#### 融合已有标签体系

In [ ]:
# 结合已有业务知识（从"RiceCooker-评论+标签.xlsx"的“三级标签（待优化用户旅程标签）”sheet中抽取出来），让GPT再生成一个融合已有知识的用户旅程阶段
ujStage = """
* Purchase
* Delivery and Installation
* Customer service
* Customer feedback
* Usage Issues
* Cleaning and maintenance"""

rspDfgUj = ask_with_history(usrp="""一些电饭锅领域的专家认为电饭锅的用户旅程包括以下阶段：
{other_stage}
请一步步分析他们的用户旅程阶段划分是否有道理？是否能够体现用户所关心的电饭锅相关的特性，服务，功能等？然后请输出你认为合理的、能准确反映用户关注点的用户旅程阶段划分。""".format(other_stage=ujStage))

这些专家提出的用户旅程阶段划分确实涵盖了一些用户在购买和使用电饭锅过程中可能会经历的重要环节。让我们逐步分析这些阶段的合理性：

1. **Purchase（购买）**：
   - 这个阶段是用户旅程的起点，用户在此阶段做出购买决策并完成交易。它是合理且必要的，因为它直接涉及用户的购买行为。

2. **Delivery and Installation（配送和安装）**：
   - 配送是用户购买后的直接后续步骤，确保用户能够顺利收到产品。安装对于某些复杂的电饭锅可能是必要的，但对于大多数电饭锅，安装并不复杂，因此这个阶段可能不需要单独列出。

3. **Customer Service（客户服务）**：
   - 客户服务在用户旅程中扮演重要角色，尤其是在用户遇到问题或需要帮助时。这个阶段的划分是合理的，因为它直接影响用户的满意度和品牌忠诚度。

4. **Customer Feedback（客户反馈）**：
   - 收集客户反馈是了解用户满意度和产品改进需求的重要途径。这个阶段的划分是合理的，因为它帮助品牌了解用户体验并进行改进。

5. **Usage Issues（使用问题）**：
   - 用户在使用过程中可能会遇到问题，这个阶段的划分是合理的，因为它涉及用户对产品功能和性能的实际体验。

6. **Cleaning and Maintenance（清洁和维护）**：
   - 电饭锅的清洁和维护是用户使用过程中的一部分，影响产品的使用寿命和用户体验。这个阶段的划分是合理的，因为它涉及用户对产品的长期使用和保养。

基于以上分析，我认为一个更全面且能准确反映用户关注点的用户旅程阶段划分可以是：

1. **Awareness and Research（意识和研究）**：
   - 用户识别需求并开始收集信息，了解不同品牌和型号的电饭锅。

2. **Purchase Decision（购买决策）**：
   - 用户比较和评估不同选项，做出购买决策并完成交易。

3. **Delivery and Setup（配送和设置）**：
   - 用户收到产品并进行必要的设置，确保电饭锅可以正常使用。

4. **Usage Experience（使用体验）**：
   - 用户开始使用电饭锅，体验其功能和性能。

5. **Customer Support（

#### 人工确定初版体系

In [7]:
# 人工调整生成的特性列表。因gpt每次生成都会略有不同，所以固定一个你认为好的即可。
# 从
""" 
1. **Awareness and Research（意识和研究）**：
   - 用户意识到需求并开始研究不同的电饭锅选项。

2. **Purchase Decision（购买决策）**：
   - 用户比较不同产品，做出购买决策。

3. **Transaction and Delivery（交易和交付）**：
   - 用户完成购买，关注支付便捷性和配送服务。

4. **Setup and Initial Use（设置和初次使用）**：
   - 用户收到产品后进行设置，了解基本功能和使用方法。

5. **Ongoing Use and Support（持续使用和支持）**：
   - 用户在日常使用中可能需要客户支持，解决使用问题。

6. **Maintenance and Longevity（维护和耐用性）**：
   - 用户关注产品的清洁、维护和耐用性。

7. **Feedback and Advocacy（反馈和倡导）**：
   - 用户分享使用体验，可能成为品牌的倡导者。
"""
# 改成（‘设置和初次使用’与‘交付’结合，把‘维护和耐用性’放到‘使用和体验’中，‘客户服务’单独拿出来）
ujStageStr = """ 
1. **Awareness and Research（意识和研究）**：
   - 用户意识到需求并开始研究不同的电饭锅选项。

2. **Consideration and Purchase（考虑与购买）**：
   - 用户在研究的基础上比较不同产品、价格、品牌等，做出购买决策。
   - 用户完成购买，关注折扣、支付便捷性和安全性等等。

3. **Delivery and Setup（交付和安装）**：
   - 交付是用户体验的重要组成部分，尤其是在电商平台购买时。用户关心的是配送速度、包装完整性和安装的简便性。
   - 对于电饭锅来说，安装通常比较简单，但用户可能需要了解如何正确使用和清洗。

4. **Ongoing Use and Maintenance（持续使用和维护）**：
   - 用户收到电饭锅后，开始使用并体验其功能。
   - 使用过程中，用户也关注产品的清洁、维护和耐用性。

5. **Customer Service（客户服务）**：
   - 优质的客户服务是用户满意度和忠诚度的重要保障。用户可能会在购买前后咨询产品信息、使用方法或解决问题。
   - 用户在日常使用中可能需要客户支持，解决使用问题。

6. **Feedback and Advocacy（反馈和倡导）**：
   - 用户分享使用体验，可能成为品牌的倡导者。
   - 用户可能会在未来再次购买同一品牌的其他产品，或推荐给朋友和家人。
"""

### GPT生成服务相关标签体系

In [8]:
# 仅基于GPT已有知识，抽取出不同用户旅程阶段中的服务类型
ask_with_history(
"""在用户旅程的不同阶段，电饭锅厂商会向用户提供不同的服务，比如配送服务、售后服务等。现在电饭锅的用户旅程阶段如下:
{ujstages}
请罗列不同阶段的厂商提供给用户的各种不同服务的类型，若某个阶段没有对应的服务类型，则在该阶段下回复**无**。""".format(ujstages=ujStageStr)
)

在电饭锅用户旅程的不同阶段，厂商可以提供以下服务：

1. **Awareness and Research（意识和研究）**：
   - **产品信息提供**：通过官网、社交媒体、广告等渠道提供详细的产品信息、功能介绍和用户评价。
   - **在线咨询服务**：提供在线客服或聊天机器人，解答用户的初步疑问。

2. **Consideration and Purchase（考虑与购买）**：
   - **促销和折扣**：提供限时折扣、优惠券或捆绑销售等促销活动。
   - **支付支持**：提供多种支付方式，确保支付过程的便捷性和安全性。
   - **购买指导**：通过客服或导购服务，帮助用户选择最适合的产品。

3. **Delivery and Setup（交付和安装）**：
   - **快速配送服务**：提供多种配送选项，包括标准配送、加急配送等。
   - **包装保护**：确保产品包装的完整性和安全性。
   - **使用指南**：提供详细的使用说明书或在线视频教程，帮助用户了解如何正确使用和清洗电饭锅。

4. **Ongoing Use and Maintenance（持续使用和维护）**：
   - **维护建议**：提供产品的清洁和维护指南，帮助用户延长产品使用寿命。
   - **产品更新通知**：通知用户关于产品的更新或升级信息。

5. **Customer Service（客户服务）**：
   - **售后支持**：提供保修服务、维修服务和更换服务。
   - **技术支持**：通过电话、邮件或在线客服提供技术支持，解决用户在使用过程中遇到的问题。
   - **用户培训**：提供使用技巧和常见问题解答，帮助用户更好地使用产品。

6. **Feedback and Advocacy（反馈和倡导）**：
   - **用户反馈收集**：通过问卷调查、在线评价等方式收集用户反馈。
   - **忠诚计划**：提供会员积分、返现或推荐奖励，鼓励用户再次购买或推荐他人。
   - **社区互动**：建立用户社区或论坛，促进用户之间的交流和分享。

通过在每个阶段提供相应的服务，厂商可以提升用户体验，增加用户满意度和品牌忠诚度。


ChatCompletion(id='chatcmpl-ASennWL6aH6sfNLHAV9IGpWgUAh78', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='在电饭锅用户旅程的不同阶段，厂商可以提供以下服务：\n\n1. **Awareness and Research（意识和研究）**：\n   - **产品信息提供**：通过官网、社交媒体、广告等渠道提供详细的产品信息、功能介绍和用户评价。\n   - **在线咨询服务**：提供在线客服或聊天机器人，解答用户的初步疑问。\n\n2. **Consideration and Purchase（考虑与购买）**：\n   - **促销和折扣**：提供限时折扣、优惠券或捆绑销售等促销活动。\n   - **支付支持**：提供多种支付方式，确保支付过程的便捷性和安全性。\n   - **购买指导**：通过客服或导购服务，帮助用户选择最适合的产品。\n\n3. **Delivery and Setup（交付和安装）**：\n   - **快速配送服务**：提供多种配送选项，包括标准配送、加急配送等。\n   - **包装保护**：确保产品包装的完整性和安全性。\n   - **使用指南**：提供详细的使用说明书或在线视频教程，帮助用户了解如何正确使用和清洗电饭锅。\n\n4. **Ongoing Use and Maintenance（持续使用和维护）**：\n   - **维护建议**：提供产品的清洁和维护指南，帮助用户延长产品使用寿命。\n   - **产品更新通知**：通知用户关于产品的更新或升级信息。\n\n5. **Customer Service（客户服务）**：\n   - **售后支持**：提供保修服务、维修服务和更换服务。\n   - **技术支持**：通过电话、邮件或在线客服提供技术支持，解决用户在使用过程中遇到的问题。\n   - **用户培训**：提供使用技巧和常见问题解答，帮助用户更好地使用产品。\n\n6. **Feedback and Advocacy（反馈和倡导）**：\n   - **用户反馈收集**：通过问卷调查、在线评价等方

In [9]:
# 人工介入，改写生成的服务类型
# 这是生成的
"""
1. **Awareness and Research（意识和研究）**：
   - **市场营销和广告**：通过广告、社交媒体、博客和视频等渠道提高品牌和产品的知名度。
   - **产品信息和比较工具**：在官网或电商平台上提供详细的产品信息、规格比较工具和用户评价。

2. **Consideration and Purchase（考虑与购买）**：
   - **促销和折扣**：提供优惠券、折扣活动或捆绑销售以吸引用户购买。
   - **安全便捷的支付选项**：提供多种支付方式，如信用卡、电子钱包等，确保支付过程的安全性和便捷性。
   - **在线客服支持**：通过在线聊天、电话或电子邮件解答用户的疑问，帮助他们做出购买决策。

3. **Delivery and Setup（交付和安装）**：
   - **快速配送服务**：提供多种配送选项，包括标准配送、加急配送等。
   - **安装指导**：提供详细的安装说明书或在线视频教程，帮助用户正确设置和初次使用电饭锅。
   - **包装完整性保障**：确保产品在运输过程中得到良好保护，避免损坏。

4. **Ongoing Use and Maintenance（持续使用和维护）**：
   - **使用指南和教程**：提供详细的使用说明书、在线教程或视频，帮助用户充分利用电饭锅的功能。
   - **维护和清洁建议**：提供产品的清洁和维护指南，帮助用户延长产品使用寿命。

5. **Customer Service（客户服务）**：
   - **售后支持**：提供保修服务、维修服务和更换服务，解决用户在使用过程中遇到的问题。
   - **客户支持渠道**：通过电话、电子邮件、在线聊天等多种渠道提供客户支持。

6. **Feedback and Advocacy（反馈和倡导）**：
   - **用户反馈收集**：通过调查问卷、用户评价平台等方式收集用户反馈。
   - **忠诚度计划**：提供会员计划、积分奖励或推荐奖励，鼓励用户再次购买或推荐给他人。
   - **社交媒体互动**：在社交媒体上与用户互动，分享用户生成的内容和评价。
"""
# 改为（去掉了一些内容）
genSrvStr = """ 
**Awareness and Research（意识和研究）**

**Consideration and Purchase（考虑与购买）**：
   - **售前客服支持**：通过在线聊天、电话或电子邮件解答用户的疑问，帮助他们做出购买决策。

**Delivery and Setup（交付和安装）**：
   - **快速配送服务**：提供多种配送选项，包括标准配送、加急配送等。同时，确保产品在运输过程中得到良好保护，避免损坏。
   - **安装服务**：提供详细的安装说明书或在线视频教程，帮助用户自己正确安装和初次使用电饭锅。甚至有专门的服务人员上门帮助安装和教导用户使用。

**Ongoing Use and Maintenance（持续使用和维护）**：
   - **使用指南和教程**：提供详细的使用说明书、在线教程或视频，帮助用户充分利用电饭锅的功能。同时也包含了产品的清洁和维护指南，帮助用户延长产品使用寿命。

**Customer Service（客户服务）**：
   - **售后客服支持**：提供保修服务、维修服务和更换服务，解决用户在使用过程中遇到的问题。

**Feedback and Advocacy（反馈和倡导）**：
   - **用户反馈收集**：通过调查问卷、用户评价平台等方式收集用户反馈。
"""


#### 融合已有标签体系

In [ ]:

# 基于已有业务知识让GPT进行调整，这是从"RiceCooker-评论+标签.xlsx"的“三级标签（待优化用户旅程标签）”sheet中抽取出来的
exlSrvStr = """ 
**Awareness and Research（意识和研究）**

**Consideration and Purchase（考虑与购买）**

**Delivery and Setup（交付和安装）**：
    - Delivery
      - Packaging
      - Protective Packaging
      - Arriving defective
      - Delivery
    - Instruction manual
      - Lack of instruction manual
      - Language problems
      - Unclear or confusing instructions
      - Incomplete information
      - Symbols and programs
      - Incorrect feature descriptions
      - Discrepancies in specifications
      - Online manual problems
      - Lack of quick-start guide
      - Safety information overemphasis
      - Electronic Manual
    - Setup
      - Setup service quality
      - Self-installation
      - Old appliance removal
      - Installation
      - Dismantling

**Ongoing Use and Maintenance（持续使用和维护）**

**Customer Service（客户服务）**：
    - Goods Return Service
    - Warranty
    - Repair Issues

**Feedback and Advocacy（反馈和倡导）**
"""

rspSrv = ask_with_history(usrp="""
# 改良后的各用户旅程阶段所涉及的服务类型

刚才生成的不同阶段的服务类型有点太多，现在将它改良如下。注意，若某个阶段下没有对应的服务类型，该阶段下则置空：
{genSrv}

# 其它电饭锅领域的电商专家提出的关于服务的标签体系

该标签体系不一定对，仅作参考。它的一级标签是用户旅程阶段，二级标签是该阶段下对应的服务类型，若某个阶段下没有对应的服务类型，则置空。三级标签是用户对该服务类型的关注点，是服务中可能导致用户满意或不满的方方面面，用户有可能表扬这些点，也有可能批评这些点。
{exlSrv}

请一步步分析上述的服务类型和标签体系的划分是否合理？结合两者的优点，以能够体现用户所关心的服务的那些方面为目的，提出改良后的关于服务的三级标签体系。

一级标签是用户旅程阶段，二级标签是该阶段下对应的服务类型，若某个阶段下没有对应的服务类型，则置空。三级标签是用户对该服务类型的关注点，是服务中可能导致用户满意或不满的方方面面，用户有可能表扬这些点，也有可能批评这些点。

请返回改良后的服务标签体系。""".format(genSrv=genSrvStr, exlSrv=exlSrvStr))

在分析上述服务类型和标签体系时，我们可以看到一些合理的划分和需要改进的地方：

1. **合理性**：
   - 各个阶段的服务类型基本涵盖了用户在电饭锅购买和使用过程中可能需要的服务。
   - 三级标签详细列出了用户可能关注的具体问题，帮助识别用户满意或不满的具体原因。

2. **需要改进的地方**：
   - 某些阶段的服务类型和关注点可以进一步细化，以更好地反映用户的实际体验。
   - 三级标签中有些关注点可能过于细碎，可以合并或简化，以便于理解和使用。

结合两者的优点，以下是改良后的服务三级标签体系：

**Awareness and Research（意识和研究）**

**Consideration and Purchase（考虑与购买）**：
   - **售前客服支持**
     - 响应速度
     - 信息准确性
     - 服务态度

**Delivery and Setup（交付和安装）**：
   - **Delivery**
     - 配送速度
     - 包装完整性
     - 运输损坏
   - **Instruction Manual**
     - 说明书清晰度
     - 语言可读性
     - 信息完整性
     - 快速入门指南
   - **Setup**
     - 自行安装难易度
     - 安装服务质量
     - 安装指导清晰度

**Ongoing Use and Maintenance（持续使用和维护）**：
   - **使用指南和教程**
     - 指南易用性
     - 教程实用性
     - 维护建议有效性

**Customer Service（客户服务）**：
   - **售后客服支持**
     - 响应速度
     - 问题解决效率
     - 保修政策透明度
   - **Goods Return Service**
     - 退货流程简便性
     - 退货政策清晰度
   - **Warranty and Repair**
     - 保修期长度
     - 维修服务质量
     - 维修时间

**Feedback and Advocacy（反馈和倡导）**：
   - **用户反馈收集**
     - 反馈渠道便捷性
 

In [11]:
# 将标签体系的最后一个层级，修改为用户对服务关注点的评价或看法
rspSrv = ask_with_history(usrp="""
# 刚生成的标签体系有一些错误。
 
刚生成的标签体系的错误是：三级标签并没体现用户对这个关注点的评价。正确的三级标签应该要体现用户对服务的关注点的好评或差评，举例如下：

**Consideration and Purchase（考虑与购买）**
   - **售前客服支持**
     - 响应速度快
     - 响应速度慢
     - 信息准确性高
     - 信息准确性低
     - 服务态度好
     - 服务态度差

# 请修正刚刚生成的标签体系的三级标签
正确的三级标签应该要体现用户对服务的关注点的好评或差评，请对上一次生成的标签体系进行修正，并输出修正后的标签体系。""")

以下是修正后的服务三级标签体系，三级标签体现了用户对服务关注点的好评或差评：

**Awareness and Research（意识和研究）**

**Consideration and Purchase（考虑与购买）**：
   - **售前客服支持**
     - 响应速度快
     - 响应速度慢
     - 信息准确性高
     - 信息准确性低
     - 服务态度好
     - 服务态度差

**Delivery and Setup（交付和安装）**：
   - **Delivery**
     - 配送速度快
     - 配送速度慢
     - 包装完整
     - 包装破损
     - 运输无损坏
     - 运输损坏
   - **Instruction Manual**
     - 说明书清晰
     - 说明书模糊
     - 语言可读性高
     - 语言可读性低
     - 信息完整
     - 信息不完整
     - 快速入门指南有用
     - 快速入门指南无用
   - **Setup**
     - 自行安装简单
     - 自行安装困难
     - 安装服务质量高
     - 安装服务质量低
     - 安装指导清晰
     - 安装指导模糊

**Ongoing Use and Maintenance（持续使用和维护）**：
   - **使用指南和教程**
     - 指南易用性高
     - 指南易用性低
     - 教程实用性强
     - 教程实用性弱
     - 维护建议有效
     - 维护建议无效

**Customer Service（客户服务）**：
   - **售后客服支持**
     - 响应速度快
     - 响应速度慢
     - 问题解决效率高
     - 问题解决效率低
     - 保修政策透明
     - 保修政策不透明
   - **Goods Return Service**
     - 退货流程简便
     - 退货流程复杂
     - 退货政策清晰
     - 退货政策模糊
   - **Warranty and Repair**
     - 保修期长
     - 保修期短
     - 维修服务质

#### 人工确定初版体系

In [12]:
# 人工再进行调整，并固定如下
srvTagStr = """
**Awareness and Research（意识和研究）**

**Consideration and Purchase（考虑与购买）**
   - **售前客服支持**
     - 响应速度快
     - 响应速度慢
     - 信息准确性高
     - 信息不准确
     - 服务态度好
     - 服务态度差
     - 问题解决能力强
     - 问题解决能力弱

**Delivery and Setup（交付和安装）**
   - **配送服务**
     - 配送速度快
     - 配送速度慢
     - 包装完整
     - 包装破损
     - 无运输损坏
     - 运输损坏
   - **安装和使用指导**
     - 说明书清晰
     - 说明书不清晰
     - 语言易懂
     - 语言难懂
     - 在线教程有用
     - 在线教程无用
     - 安装服务质量高
     - 安装服务质量低
     - 自行安装方便
     - 自行安装困难

**Ongoing Use and Maintenance（持续使用和维护）**
   - **使用指南和教程**
     - 功能说明清晰
     - 功能说明不清晰
     - 清洁和维护指导详细
     - 清洁和维护指导不足
     - 在线资源丰富
     - 在线资源匮乏

**Customer Service（客户服务）**
   - **售后支持**
     - 保修政策合理
     - 保修政策不合理
     - 维修服务响应快
     - 维修服务响应慢
     - 更换服务高效
     - 更换服务低效

   - **退货服务**
     - 退货流程简便
     - 退货流程复杂
     - 退款速度快
     - 退款速度慢

**Feedback and Advocacy（反馈和倡导）**
   - **用户反馈收集**
     - 反馈渠道多样
     - 反馈渠道单一
     - 反馈响应快
     - 反馈响应慢
"""
# *** 通过gpt将上面列表转成三层dict
srvTagDict = {  
    "Awareness and Research（意识和研究）": {},  
    "Consideration and Purchase（考虑与购买）": {  
        "售前客服支持": {  
            "响应速度快",  
            "响应速度慢",  
            "信息准确性高",  
            "信息不准确",  
            "服务态度好",  
            "服务态度差",  
            "问题解决能力强",  
            "问题解决能力弱",  
        }  
    },  
    "Delivery and Setup（交付和安装）": {  
        "配送服务": {  
            "配送速度快",  
            "配送速度慢",  
            "包装完整",  
            "包装破损",  
            "无运输损坏",  
            "运输损坏",  
        },  
        "安装和使用指导": {  
            "说明书清晰",  
            "说明书不清晰",  
            "语言易懂",  
            "语言难懂",  
            "在线教程有用",  
            "在线教程无用",  
            "安装服务质量高",  
            "安装服务质量低",  
            "自行安装方便",  
            "自行安装困难",  
        }  
    },  
    "Ongoing Use and Maintenance（持续使用和维护）": {  
        "使用指南和教程": {  
            "功能说明清晰",  
            "功能说明不清晰",  
            "清洁和维护指导详细",  
            "清洁和维护指导不足",  
            "在线资源丰富",  
            "在线资源匮乏",  
        }  
    },  
    "Customer Service（客户服务）": {  
        "售后支持": {  
            "保修政策合理",  
            "保修政策不合理",  
            "维修服务响应快",  
            "维修服务响应慢",  
            "更换服务高效",  
            "更换服务低效",  
        },  
        "退货服务": {  
            "退货流程简便",  
            "退货流程复杂",  
            "退款速度快",  
            "退款速度慢",  
        }  
    },  
    "Feedback and Advocacy（反馈和倡导）": {  
        "用户反馈收集": {  
            "反馈渠道多样",  
            "反馈渠道单一",  
            "反馈响应快",  
            "反馈响应慢",  
        }  
    }  
}  

### GPT生成产品特性相关标签体系

In [13]:
# 仅用GPT生成电饭锅产品特性相关的标签
rspFeas = ask_with_history(usrp="""用户在选择和评论电饭锅时，要考虑到电饭锅的方方面面，比如特性、功能、部件、外观、使用便捷，烹饪效果等等。
请你生成一个二级标签体系，要体现上述用户选择和评价电饭锅时要考虑的方方面面，注意，不要包含服务相关的特性，而且一级标签前不要有数字顺序标记。""")

以下是一个二级标签体系，体现了用户在选择和评价电饭锅时要考虑的各个方面，不包含服务相关的特性：

**特性与功能**
   - 多功能性
   - 烹饪模式
   - 定时功能
   - 保温功能
   - 快速烹饪
   - 能耗效率

**部件与材料**
   - 内胆材质
   - 加热技术
   - 盖子设计
   - 控制面板
   - 附件质量

**外观与设计**
   - 颜色选择
   - 体积大小
   - 设计风格
   - 显示屏设计
   - 便携性

**使用便捷性**
   - 操作简便性
   - 清洗难易度
   - 说明书易读性
   - 维护便捷性

**烹饪效果**
   - 米饭口感
   - 烹饪均匀性
   - 烹饪速度
   - 保温效果

这个标签体系帮助用户在选择和评价电饭锅时，全面考虑产品的各个方面，从而做出更明智的决策。


#### 融合已有标签体系

In [ ]:
# 结合已有的业务知识（从"RiceCooker-评论+标签.xlsx"的“三级标签（待优化用户旅程标签）”sheet中抽取来）
exlFeaStr=""" 
**Brand**
  - Brand
**Price**
  - Promotional
  - Value for money
  - expensive
**Capacity**
  - Small interior capacity
  - Capacity
**Size**
  - Suitable for small households
  - Installation and Fit
  - Portable and flexible
  - Space saving design
  - Compact for small spaces
**Design**
  - Appearance
  - Interior Layout
  - Innovation
  - Shape
  - Height
**Color**
  - Color
**Weight**
  - Weight
**Accessories**
  - Missing Accessories
  - Additional Purchases Required
  - Quality  Concerns
  - Accessories compatibility issues
  - Specific Accessory Requests
  - Power Adapter Problems
  - Pleasantly surprised with unexpected accessories
  - Replacement Parts
  - Setup accessories
**Quality**
  - Broken Item
  - Materials quality
  - Product life
  - Product durability
  - Scratched
  - Dent
**Device failure**
  - Failure to Start or Operate
  - Component malfunctions
  - Device overheating
  - shutting off unexpectedly
  - Power supply problems
  - Sensor malfunctions
  - Defective Units
  - breakdown after a short use
  - Buttons
  - Error codes
  - Sparking or catching fire
**Control panel**
  - Complexity
  - Visibility 
  - Imprecise controls
  - Buttons or knobs breaking
  - Detached from the device
  - Wear and tear on button surfaces
  - Language barriers
  - Not suitable for visually impaired users
  - Software glitches
  - Analog vs. digital 
**Lighting**
  - Dim interior light
  - Display brightness
  - Indicator Light Not On
**Timer**
  - Timer inaccuracies
  - Difficulty in setting time
  - Display problems
  - Reset issues
  - Analog vs. digital preferences
**Odor**
  - plastic or chemical smell
  - Rubber smell
  - Burning or smoke smell
  - Persistent odors
  - Humid or musty smells
**Noise**
  - Quiet
  - Loud noise
  - General noise level
  - Specific components
  - Alarm sounds
  - End-of-cycle signals
  - Noise over time
  - Eco mode
  - Unexpected noises
  - Vibration noises
  - Grinding noises
  - Beep Sound
**IOT**
  - WIFI connection
  - App Control
  - Program Customization
  - Maintenance Alerts
  - Language and Translation Issues
  - Smart Home Integration
  - Troubleshooting
  - Push Notifications
**Energy efficiency**
  - Energy consumption
  - Eco Programs
  - Comparing to advertised specifications
  - Performance in different settings
  - Justifying purchase decisions
  - Specific Energy-Saving Technologies
  - Providing usage tips
**Cleaning and maintenance**
  - Cleanliness
  - Odor retention
  - Self-cleaning function
  - Maintenance
  - Exterior surfaces cleaning
  - Dirtiness
  - Easy to Clean
**Coating**
  - Coating Peeling Off
  - Coating Scratched
  - Coating Bubbling
**Safety**
  - High Temperature Burns
  - Fire / Self-Ignition
  - Electric Leakage
  - Smoking
  - Explosion
**Aging**
  - Aging
**Workmanship**
  - Workmanship
**Discoloration**
  - Discoloration
**Material**
  - Material
**Power On/Off Issues**
  - Not Working When Powered
  - Not Powering On
**Button **
  - Button Failure
  - Button  Sensitive
**Knob **
  - Knob Stuck
  - Knob Operation Smooth
  - Knob Sensitive
**Cooking Effect**
  - Cooking speed
  - Water-to-rice ratio
  - Cooking different rice varieties
  - Versatility beyond rice
  - sticking to the pot
  - Even cooking and preventing burning
  - Soaking and pre-cooking
  - adjusting cooking times
**Insulation Effect**
  - Keep warm duration
  - Drying out on keep warm
  - Inconsistent warming
**Steaming Effect**
  - Versatility for steaming
  - Steam venting
  - Steaming performance
**Hot to Touch**
  - Hot to Touch
**Steaming Basket**
  - lack of Steaming Basket
  -  steamer basket too small
**Steaming Rack**
  - Steaming Rack
**Inner Pot**
  - Inner Pot
**Lid**
  - Opening Lid
  - Closing Lid
**Measuring Cup**
  - Measuring Cup
**Spoon**
  - Spoon
**Steam Valve**
  - Steam Valve
**Rubber Pad**
  - Rubber Pad
**Sealing Ring**
  - Sealing Ring
**Water Storage Box**
  - Water Storage Box
**Nameplate**
  - Nameplate
**Screws**
  - Screws
**Handling**
  - Handling
**Rust**
  - Screws Rust
**Taste**
  - Rice Texture
**QR Code**
  - QR Code
**Soup Cooking**
  - Good Soup Cooking Result
  - Fast Soup Cooking
**Rice Cooking**
  - Good Rice Cooking Result
  - Fast Rice Cooking
  - Long Cooking Time 
  - Crust Formation
  - Rice Overflow
  - Rice Undercooked
  - Rice Too Hard
  - Rice Too Mushy
  - Not Fluffy
  - No Rice Fragrance
  - Not Sweet Enough
**Porridge Cooking**
  - Good Porridge Cooking Result
  - Porridge Overflow
  - Porridge Undercooked
  - Long Cooking Time for Porridge
  - Not Mushy Enough
  - Fast Porridge Cooking
**water leakage**
  - Spraying water
  - Leaking from lids/seals
  - Overflowing/Boiling over
  - Condensation accumulation
  - Leaking from bottoms
  - Counter/floor soaking
  - Lack of drainage
  - Drainage cup design
**Power Cord**
  - Power Cord
**Storage**
  - Storage
**Limited functionality**
  - Limited functionality
  - Lack of automatic functions
"""

In [15]:
# 结合已有业务知识，生成新的产品特性标签体系
rspPrdFea = ask_with_history(usrp="""
# 刚生成的标签体系有一些错误。
 
刚生成的标签体系的错误是：有一些一级标签的粒度比较粗，需要更细一些。另外，正确的二级标签应该要体现用户对电饭锅的各个关注点、各个功能特性等的好评或差评，比如颜色可以从二级标签改为一级标签，举例如下：

**颜色**
    - 颜色不好看
    - 颜色好看
    - 颜色单一
    - 颜色不协调
    - 颜色耐脏
    - 颜色不耐脏
    - 其它

# 其它的专家做出他生成的关于电饭锅产品各种特性、功能、等方方面面的标签体现，标签的粗细粒度还可以，可供参考。
{exlPrdFeas}

# 刚刚生成你一级标签粒度过粗的标签体系，请修正刚生成的标签体系，使得一级标签粒度不要太粗，同时二级标签要体现用户对电饭锅的各个关注点、各个功能特性等的好评或差评

请融合其它专家作出的标签体系（一级标签最好尽可能结合他的，或在他的基础上扩充，但如果实在不合理，也可以不用），对上一次生成的标签体系进行修正，并输出修正后的中文标签体系。""".format(exlPrdFeas=exlFeaStr))

以下是修正后的中文标签体系，结合了其他专家的标签体系，并细化了一级标签，同时二级标签体现了用户对电饭锅各个关注点的好评或差评：

**品牌**
   - 品牌知名度高
   - 品牌知名度低

**价格**
   - 性价比高
   - 性价比低
   - 价格昂贵
   - 价格实惠

**容量**
   - 容量适中
   - 容量过小
   - 容量过大

**尺寸**
   - 适合小家庭
   - 设计紧凑
   - 占用空间小
   - 便携性强

**设计**
   - 外观美观
   - 外观普通
   - 创新设计
   - 设计过时

**颜色**
   - 颜色好看
   - 颜色不好看
   - 颜色耐脏
   - 颜色不耐脏

**重量**
   - 重量轻
   - 重量重

**配件**
   - 配件齐全
   - 配件缺失
   - 配件质量高
   - 配件质量低

**质量**
   - 材质优良
   - 材质一般
   - 产品耐用
   - 产品易损

**设备故障**
   - 设备故障少
   - 设备故障多

**控制面板**
   - 操作简单
   - 操作复杂
   - 按钮耐用
   - 按钮易损

**照明**
   - 显示屏亮度高
   - 显示屏亮度低

**定时器**
   - 定时准确
   - 定时不准

**气味**
   - 无异味
   - 异味明显

**噪音**
   - 噪音小
   - 噪音大

**物联网功能**
   - 连接稳定
   - 连接不稳定

**能效**
   - 能耗低
   - 能耗高

**清洁与维护**
   - 易于清洁
   - 难以清洁

**涂层**
   - 涂层耐用
   - 涂层易损

**安全性**
   - 安全性高
   - 安全性低

**老化**
   - 老化慢
   - 老化快

**做工**
   - 做工精细
   - 做工粗糙

**变色**
   - 不易变色
   - 易变色

**材料**
   - 材料优质
   - 材料一般

**开关问题**
   - 开关灵敏
   - 开关不灵敏

**按钮**
   - 按钮灵敏
   - 按钮不灵敏

**旋钮**
   - 旋钮操作顺畅
   - 旋钮操作困难

**

In [16]:
# 修剪输出格式
rspPrdFea = ask_with_history("请仅返回上述的电饭锅产品特性的标签体系，不包含其它内容。")

**品牌**
   - 品牌知名度高
   - 品牌知名度低

**价格**
   - 性价比高
   - 性价比低
   - 价格昂贵
   - 价格实惠

**容量**
   - 容量适中
   - 容量过小
   - 容量过大

**尺寸**
   - 适合小家庭
   - 设计紧凑
   - 占用空间小
   - 便携性强

**设计**
   - 外观美观
   - 外观普通
   - 创新设计
   - 设计过时

**颜色**
   - 颜色好看
   - 颜色不好看
   - 颜色耐脏
   - 颜色不耐脏

**重量**
   - 重量轻
   - 重量重

**配件**
   - 配件齐全
   - 配件缺失
   - 配件质量高
   - 配件质量低

**质量**
   - 材质优良
   - 材质一般
   - 产品耐用
   - 产品易损

**设备故障**
   - 设备故障少
   - 设备故障多

**控制面板**
   - 操作简单
   - 操作复杂
   - 按钮耐用
   - 按钮易损

**照明**
   - 显示屏亮度高
   - 显示屏亮度低

**定时器**
   - 定时准确
   - 定时不准

**气味**
   - 无异味
   - 异味明显

**噪音**
   - 噪音小
   - 噪音大

**物联网功能**
   - 连接稳定
   - 连接不稳定

**能效**
   - 能耗低
   - 能耗高

**清洁与维护**
   - 易于清洁
   - 难以清洁

**涂层**
   - 涂层耐用
   - 涂层易损

**安全性**
   - 安全性高
   - 安全性低

**老化**
   - 老化慢
   - 老化快

**做工**
   - 做工精细
   - 做工粗糙

**变色**
   - 不易变色
   - 易变色

**材料**
   - 材料优质
   - 材料一般

**开关问题**
   - 开关灵敏
   - 开关不灵敏

**按钮**
   - 按钮灵敏
   - 按钮不灵敏

**旋钮**
   - 旋钮操作顺畅
   - 旋钮操作困难

**烹饪效果**
   - 烹饪速度快
   - 烹饪速度慢
   - 米饭口感好
   - 米饭口感差

**保温效果**
   -

#### 人工确定产品特性标签体系

In [17]:
# 人工调整
prdFeaStr = rspPrdFea.choices[0].message.content.strip()
# print(prdFeaStr)
prdFeaStr = """ 
**品牌**
   - 品牌知名度高
   - 品牌知名度低

**价格**
   - 性价比高
   - 性价比低
   - 价格昂贵
   - 价格实惠

**容量**
   - 容量适中
   - 容量过小
   - 容量过大

**尺寸**
   - 适合小家庭
   - 设计紧凑
   - 占用空间小
   - 便携性强

**设计**
   - 外观美观
   - 外观不美观
   - 创新设计
   - 设计传统

**颜色**
   - 颜色好看
   - 颜色不好看
   - 颜色耐脏
   - 颜色不耐脏

**重量**
   - 重量轻
   - 重量重

**配件**
   - 配件齐全
   - 配件缺失
   - 配件质量高
   - 配件质量低

**质量**
   - 材质优良
   - 材质劣质
   - 产品耐用
   - 产品易损

**设备故障**
   - 设备故障频繁
   - 设备运行稳定

**控制面板**
   - 操作简单
   - 操作复杂
   - 显示清晰
   - 显示模糊

**照明**
   - 照明充足
   - 照明不足

**定时器**
   - 定时准确
   - 定时不准

**气味**
   - 无异味
   - 有异味

**噪音**
   - 噪音小
   - 噪音大

**物联网功能**
   - 连接稳定
   - 连接不稳定

**能效**
   - 能耗低
   - 能耗高

**清洁与维护**
   - 易于清洁
   - 难以清洁

**涂层**
   - 涂层耐用
   - 涂层易脱落

**安全性**
   - 安全性高
   - 安全隐患

**老化**
   - 老化速度慢
   - 老化速度快

**做工**
   - 做工精细
   - 做工粗糙

**变色**
   - 不易变色
   - 易变色

**材料**
   - 材料环保
   - 材料不环保

**开关问题**
   - 开关灵敏
   - 开关失灵

**按钮**
   - 按钮灵敏
   - 按钮失灵

**旋钮**
   - 旋钮操作顺畅
   - 旋钮卡顿

**烹饪功能**
   - 烹饪速度快
   - 烹饪速度慢

**保温效果**
   - 保温持久
   - 保温效果差

**蒸煮效果**
   - 蒸煮均匀
   - 蒸煮不均匀

**触感**
   - 外壳不烫手
   - 外壳烫手

**蒸笼**
   - 蒸笼容量大
   - 蒸笼容量小

**内胆**
   - 内胆耐用
   - 内胆易损

**盖子**
   - 盖子易开合
   - 盖子难开合

**量杯**
   - 量杯准确
   - 量杯不准

**勺子**
   - 勺子实用
   - 勺子不实用

**蒸汽阀**
   - 蒸汽阀安全
   - 蒸汽阀不安全

**橡胶垫**
   - 橡胶垫耐用
   - 橡胶垫易损

**密封圈**
   - 密封圈密封性好
   - 密封圈漏气

**水箱**
   - 水箱容量大
   - 水箱容量小

**铭牌**
   - 铭牌清晰
   - 铭牌模糊

**螺丝**
   - 螺丝不生锈
   - 螺丝生锈

**手柄**
   - 手柄舒适
   - 手柄不舒适

**生锈**
   - 不易生锈
   - 易生锈

**味道**
   - 米饭香味浓
   - 米饭无香味

**二维码**
   - 二维码易扫描
   - 二维码难扫描

**煮汤**
   - 煮汤效果好
   - 煮汤效果差

**煮饭**
   - 煮饭效果好
   - 煮饭效果差
   - 米饭口感好
   - 米饭口感差

**煮粥**
   - 煮粥效果好
   - 煮粥效果差

**漏水**
   - 不漏水
   - 漏水

**电源线**
   - 电源线长度合适
   - 电源线长度不合适

**存储**
   - 存储方便
   - 存储不便

**功能数量**
   - 功能多
   - 功能有限
"""

## GPT生成评论并补充新标签

通过让GPT边生成评论，边打标的方式，发掘新的标签，以补充标签体系。这里只是个例子，实际应用时需要尝试更多的角色，更多样和细致的对生成目标的描述，以及不要太多的引导标签体系。

In [18]:
# 定义角色，替换原来的system prompt
usrProfStr = "你是一个三口之家的家庭主妇，你已经用了多年的电饭锅。"
history[0] = {"role": "system", "content": usrProfStr}

synsz_rev_tmp = """{usrprofile} 请参考亚马逊网站上的用户评论风格，请生成10个用英文写的用户评论，生成的评论必须满足以下要求。

# 生成的用户评论所属的用户旅程应该是下述用户旅程阶段（UserJourneyStage）中的一个：

{ujstages}

# 生成的用户评论应该体现了表扬、批评这两种情感（Sentiments）中的一种或多种情感。

# 生成的用户评论可以覆盖一个或多个考虑点或电饭锅的各个特性（Features），所涉及的关注点或功能特性以电饭锅特性标签体系中的层级关系路径（即“父特性/子特性”）来表示。电饭锅考虑点和功能特性的标签体系如下，你最好生成新的不限于已存在标签体系中的考虑点和功能特性，而尽可能生成包含新考虑点或电饭锅功能特性（NewFeatures）的评论。下面电饭锅特性标签体系的一级标签是选择和评价电饭锅时索要考虑的点或方面、功能特性、部件、效果等，二级标签用户在选择或使用电饭锅后可能对某些产品功能特性做出的评价。

{feas}

# 如果新的考虑点或特性（NewFeatures）不是任何已有考虑点或特性的子特性的话，直接将新特性记录在'NewFeatures'中即可。但若新考虑点或特性是属于已有参考考虑点或特性（Features）的子特性的，此时请用“父特性/子特性”的格式给予记录。

# 生成的用户评论可以选择性覆盖一个或多个电饭锅厂商服务的关注点（Services），所涉及的服务及其关注点以电饭锅服务关注点标签体系中的层级关系路径（即“所属用户旅程阶段/父服务类型/子服务关注点”）来表示。电饭锅服务关注点标签体系如下，你最好生成新的不限于已存在标签体系中的服务关注点，而尽可能生成包含新服务关注点（NewServices）的评论。下述服务关注点体系的一级标签是服务的所属用户旅程阶段（一般和生成的评论的所属用户旅程阶段一致），二级标签是服务类型，三级标签是用户在使用服务时可能对某系服务关注点的评论。

{srvs}

# 如果新的服务关注点（NewServices）的父标签可以是已有服务关注点标签体系中的一级标签或二级标签，也可以是某个用户旅程阶段中的新服务类型，无论怎样都用“所属用户旅程阶段/父服务类型/子服务关注点”的格式给予记录。

# 生成的用户评论以json格式返回。说明如下：

{{
  'UserJourneyStage': 生成的用户评论是用户在哪个用户旅程阶段所产生的频率。
  'Sentiments': 生成的用户评论是对正面的点赞或是对负面的批评，还是两者都有，多个情感用逗号分隔。
  'Features': 生成的用户评论若涉及一些在电饭锅特性标签体系中已有的标签则填到这里，多个特性用逗号分隔,
  'NewFeatures': 生成的用户评论若涉及了一些在电饭锅特性标签体系中没有的特性，则以“父特性/子特性”的格式记录在这里，如果有多个则以逗号分隔。
  'Services': 生成的用户评论若涉及一些在服务关注点标签体系中已有的标签则填到这里，多个服务关注点用逗号分隔,
  'NewServices': 生成的用户评论若涉及了一些在服务关注点标签体系中没有的服务关注点，则以“所属用户旅程阶段/父服务类型/子服务关注点”的格式记录在这里，如果有多个则以逗号分隔。
  'Review': 最终生成的亚马逊风格的英文用户评论。
}},

# 生成的用户评论以json格式返回。举例如下：

[
{{
  'UserJourneyStage':'Awareness and Research（意识和研究）',
  'Sentiments':'positive',
  'Features':'清洁与维护/易于清洁,烹饪效果/烹饪速度快',
  'NewFeatures':'内胆/拆卸容易',
  'Services': '',
  'NewServices': '',
  'Review':'I want to buy a rice cooker.  And I asked my friend for suggestion.  He says that the best features of his Tiger rice cooker are the detachable inner pot and the quick-cook function. For him as a single person, the detachable inner pot makes it easy to clean the rice cooker, and the quick-cook function shortens his cooking time.'
}},
{{
  "UserJourneyStage":"Ongoing Use and Maintenance（持续使用和维护）",
  "Sentiments":"positive",
  "Features":"价格/性价比高,煮饭/煮饭效果好",
  "NewFeatures":"",
  "Services": "",
  "NewServices": "",
  "Review":"I use this product to make rice. The rice is better than the other cheaper rice cookers. I think it‘s worth the price."
}},
{{
    "UserJourneyStage": "Customer Service（客户服务）",
    "Sentiments": "negative",
    "Features": "开关问题/开关失灵",
    "NewFeatures": "",
    "Services": "Customer Service（客户服务）/售后支持/更换服务低效",
    "NewServices": "Customer Service（客户服务）/售后支持/售后服务响应慢",
    "Review": "I had an issue with the rice cooker not turning on. Customer support was slow to respond, and it took several days to get a replacement."
}},
]

# 举例结束。

请根据上述要求，并参考亚马逊网站上的用户评论风格，请生成10个用英文写的用户评论，以json格式返回。"""

synsz_rev_prmp = synsz_rev_tmp.format(usrprofile=usrProfStr, ujstages=ujStageStr, feas=prdFeaStr, srvs=srvTagStr)
rspSynszRev0 = ask_with_history(synsz_rev_prmp) 
#@#???"NewFeatures": "NewFeatures/多功能性"

```json
[
    {
        "UserJourneyStage": "Awareness and Research（意识和研究）",
        "Sentiments": "positive",
        "Features": "品牌/品牌知名度高,设计/外观美观",
        "NewFeatures": "功能数量/功能多",
        "Services": "",
        "NewServices": "",
        "Review": "I've been researching rice cookers and found that the Zojirushi brand is highly recommended. Its sleek design and multiple cooking functions make it a top contender for my purchase."
    },
    {
        "UserJourneyStage": "Consideration and Purchase（考虑与购买）",
        "Sentiments": "positive",
        "Features": "价格/性价比高,容量/容量适中",
        "NewFeatures": "",
        "Services": "Consideration and Purchase（考虑与购买）/售前客服支持/信息准确性高",
        "NewServices": "",
        "Review": "After comparing several models, I decided on this rice cooker because it offers great value for money and the perfect capacity for my family. The customer service was very helpful in providing accurate information."
    },
    {
        "UserJourneyStage": "Deliver

In [19]:
history = history[:-2] #清楚上一轮评论生成的历史，避免对下一轮另一个角色的评论生成产生影响
history

[{'role': 'system', 'content': '你是一个三口之家的家庭主妇，你已经用了多年的电饭锅。'},
 {'role': 'user', 'content': '请详细解释一下用户旅程。'},
 {'role': 'assistant',
  'content': '用户旅程是指用户从意识到某个产品或服务的需求开始，到最终购买并使用该产品或服务的整个过程。在电饭锅领域，用户旅程可以分为以下几个阶段：\n\n1. **意识阶段**：\n   - **需求识别**：用户意识到需要一个新的电饭锅，可能是因为旧的电饭锅坏了，或者他们想要一个功能更先进的电饭锅。\n   - **品牌认知**：用户开始接触到不同品牌的电饭锅，可能通过广告、朋友推荐或社交媒体。\n\n2. **考虑阶段**：\n   - **信息收集**：用户开始在网上搜索电饭锅的相关信息，包括品牌、型号、功能、价格等。他们可能会访问电商平台、品牌官网、产品评测网站等。\n   - **比较与评估**：用户将不同品牌和型号的电饭锅进行比较，评估它们的功能、价格、用户评价等，以缩小选择范围。\n\n3. **决策阶段**：\n   - **产品选择**：用户决定购买某一品牌和型号的电饭锅。\n   - **购买渠道选择**：用户选择购买渠道，可能是通过电商平台、品牌官网或线下实体店。\n\n4. **购买阶段**：\n   - **下单与支付**：用户在选择的渠道下单并完成支付。\n   - **配送与交付**：用户等待电饭锅的配送，并最终收到产品。\n\n5. **使用阶段**：\n   - **产品使用**：用户开始使用电饭锅，体验其功能和性能。\n   - **售后服务**：如果用户遇到问题，他们可能会联系售后服务寻求帮助。\n\n6. **忠诚阶段**：\n   - **满意度评估**：用户评估对电饭锅的满意度，决定是否会再次购买该品牌的产品或推荐给他人。\n   - **反馈与评价**：用户可能会在电商平台或社交媒体上留下产品评价，分享使用体验。\n\n在整个用户旅程中，品牌和商家可以通过提供优质的产品信息、良好的客户服务和积极的用户互动来影响用户的决策过程，提高用户的购买意愿和忠诚度。'},
 {'role': 'user',
  'content': '一些电饭锅领域的专家认为

In [20]:
usrProfStr = "你是一个单身的职业男性，你已经用了多年的电饭锅，期望电饭锅是易用且节省时间的。"
history[0] = {"role": "system", "content": usrProfStr}

synsz_rev_tmp = """{usrprofile} 请参考亚马逊网站上的用户评论风格，请生成10个用英文写的用户评论，生成的评论必须满足以下要求。

# 生成的用户评论所属的用户旅程应该是下述用户旅程阶段（UserJourneyStage）中的一个：

{ujstages}

# 生成的用户评论应该体现了表扬、批评这两种情感（Sentiments）中的一种或多种情感。

# 生成的用户评论可以覆盖一个或多个考虑点或电饭锅的各个特性（Features），所涉及的关注点或功能特性以电饭锅特性标签体系中的层级关系路径（即“父特性/子特性”）来表示。电饭锅考虑点和功能特性的标签体系如下，你最好生成新的不限于已存在标签体系中的考虑点和功能特性，而尽可能生成包含新考虑点或电饭锅功能特性（NewFeatures）的评论。下面电饭锅特性标签体系的一级标签是选择和评价电饭锅时索要考虑的点或方面、功能特性、部件、效果等，二级标签用户在选择或使用电饭锅后可能对某些产品功能特性做出的评价。

{feas}

# 如果新的考虑点或特性（NewFeatures）不是任何已有考虑点或特性的子特性的话，直接将新特性记录在'NewFeatures'中即可。但若新考虑点或特性是属于已有参考考虑点或特性（Features）的子特性的，此时请用“父特性/子特性”的格式给予记录。

# 生成的用户评论可以选择性覆盖一个或多个电饭锅厂商服务的关注点（Services），所涉及的服务及其关注点以电饭锅服务关注点标签体系中的层级关系路径（即“所属用户旅程阶段/父服务类型/子服务关注点”）来表示。电饭锅服务关注点标签体系如下，你最好生成新的不限于已存在标签体系中的服务关注点，而尽可能生成包含新服务关注点（NewServices）的评论。下述服务关注点体系的一级标签是服务的所属用户旅程阶段（一般和生成的评论的所属用户旅程阶段一致），二级标签是服务类型，三级标签是用户在使用服务时可能对某系服务关注点的评论。

{srvs}

# 如果新的服务关注点（NewServices）的父标签可以是已有服务关注点标签体系中的一级标签或二级标签，也可以是某个用户旅程阶段中的新服务类型，无论怎样都用“所属用户旅程阶段/父服务类型/子服务关注点”的格式给予记录。

# 生成的用户评论以json格式返回。说明如下：

{{
  'UserJourneyStage': 生成的用户评论是用户在哪个用户旅程阶段所产生的频率。
  'Sentiments': 生成的用户评论是对正面的点赞或是对负面的批评，还是两者都有，多个情感用逗号分隔。
  'Features': 生成的用户评论若涉及一些在电饭锅特性标签体系中已有的标签则填到这里，多个特性用逗号分隔,
  'NewFeatures': 生成的用户评论若涉及了一些在电饭锅特性标签体系中没有的特性，则以“父特性/子特性”的格式记录在这里，如果有多个则以逗号分隔。
  'Services': 生成的用户评论若涉及一些在服务关注点标签体系中已有的标签则填到这里，多个服务关注点用逗号分隔,
  'NewServices': 生成的用户评论若涉及了一些在服务关注点标签体系中没有的服务关注点，则以“所属用户旅程阶段/父服务类型/子服务关注点”的格式记录在这里，如果有多个则以逗号分隔。
  'Review': 最终生成的亚马逊风格的英文用户评论。
}},

# 生成的用户评论以json格式返回。举例如下：

[
{{
  'UserJourneyStage':'Awareness and Research（意识和研究）',
  'Sentiments':'positive',
  'Features':'清洁与维护/易于清洁,烹饪效果/烹饪速度快',
  'NewFeatures':'内胆/拆卸容易',
  'Services': '',
  'NewServices': '',
  'Review':'I want to buy a rice cooker.  And I asked my friend for suggestion.  He says that the best features of his Tiger rice cooker are the detachable inner pot and the quick-cook function. For him as a single person, the detachable inner pot makes it easy to clean the rice cooker, and the quick-cook function shortens his cooking time.'
}},
{{
  "UserJourneyStage":"Ongoing Use and Maintenance（持续使用和维护）",
  "Sentiments":"positive",
  "Features":"价格/性价比高,煮饭/煮饭效果好",
  "NewFeatures":"",
  "Services": "",
  "NewServices": "",
  "Review":"I use this product to make rice. The rice is better than the other cheaper rice cookers. I think it‘s worth the price."
}},
{{
    "UserJourneyStage": "Customer Service（客户服务）",
    "Sentiments": "negative",
    "Features": "开关问题/开关失灵",
    "NewFeatures": "",
    "Services": "Customer Service（客户服务）/售后支持/更换服务低效",
    "NewServices": "Customer Service（客户服务）/售后支持/售后服务响应慢",
    "Review": "I had an issue with the rice cooker not turning on. Customer support was slow to respond, and it took several days to get a replacement."
}},
]

# 举例结束。

请根据上述要求，并参考亚马逊网站上的用户评论风格，请生成10个用英文写的用户评论，以json格式返回。"""

synsz_rev_prmp = synsz_rev_tmp.format(usrprofile=usrProfStr, ujstages=ujStageStr, feas=prdFeaStr, srvs=srvTagStr)
rspSynszRev1 = ask_with_history(synsz_rev_prmp) 
#@#???"NewFeatures": "NewFeatures/多功能性"

Here are 10 user reviews in JSON format, following the specified requirements:

```json
[
    {
        "UserJourneyStage": "Awareness and Research（意识和研究）",
        "Sentiments": "positive",
        "Features": "品牌/品牌知名度高,设计/外观美观",
        "NewFeatures": "功能数量/多功能",
        "Services": "",
        "NewServices": "",
        "Review": "While researching rice cookers, I found that the Zojirushi brand is highly recommended for its sleek design and multifunctionality. It's definitely on my shortlist!"
    },
    {
        "UserJourneyStage": "Consideration and Purchase（考虑与购买）",
        "Sentiments": "positive",
        "Features": "价格/性价比高,容量/容量适中",
        "NewFeatures": "",
        "Services": "Consideration and Purchase（考虑与购买）/售前客服支持/信息准确性高",
        "NewServices": "",
        "Review": "I decided to purchase this rice cooker because it offers great value for money and the capacity is just right for my needs. The customer service was very informative and helped me make the decision."
  

In [21]:
history = history[:-2] #回滚历史
history[0] = {"role": "system", "content": sysp_default} # 重置system prompt
history

[{'role': 'system', 'content': '你是一个电饭锅领域的国际电商专家。'},
 {'role': 'user', 'content': '请详细解释一下用户旅程。'},
 {'role': 'assistant',
  'content': '用户旅程是指用户从意识到某个产品或服务的需求开始，到最终购买并使用该产品或服务的整个过程。在电饭锅领域，用户旅程可以分为以下几个阶段：\n\n1. **意识阶段**：\n   - **需求识别**：用户意识到需要一个新的电饭锅，可能是因为旧的电饭锅坏了，或者他们想要一个功能更先进的电饭锅。\n   - **品牌认知**：用户开始接触到不同品牌的电饭锅，可能通过广告、朋友推荐或社交媒体。\n\n2. **考虑阶段**：\n   - **信息收集**：用户开始在网上搜索电饭锅的相关信息，包括品牌、型号、功能、价格等。他们可能会访问电商平台、品牌官网、产品评测网站等。\n   - **比较与评估**：用户将不同品牌和型号的电饭锅进行比较，评估它们的功能、价格、用户评价等，以缩小选择范围。\n\n3. **决策阶段**：\n   - **产品选择**：用户决定购买某一品牌和型号的电饭锅。\n   - **购买渠道选择**：用户选择购买渠道，可能是通过电商平台、品牌官网或线下实体店。\n\n4. **购买阶段**：\n   - **下单与支付**：用户在选择的渠道下单并完成支付。\n   - **配送与交付**：用户等待电饭锅的配送，并最终收到产品。\n\n5. **使用阶段**：\n   - **产品使用**：用户开始使用电饭锅，体验其功能和性能。\n   - **售后服务**：如果用户遇到问题，他们可能会联系售后服务寻求帮助。\n\n6. **忠诚阶段**：\n   - **满意度评估**：用户评估对电饭锅的满意度，决定是否会再次购买该品牌的产品或推荐给他人。\n   - **反馈与评价**：用户可能会在电商平台或社交媒体上留下产品评价，分享使用体验。\n\n在整个用户旅程中，品牌和商家可以通过提供优质的产品信息、良好的客户服务和积极的用户互动来影响用户的决策过程，提高用户的购买意愿和忠诚度。'},
 {'role': 'user',
  'content': '一些电饭锅领域的专家认为电饭锅的用户旅程包

### 从生成评论中抽取标签

#### 抽取新的产品特性标签

In [22]:
# 将用于prompt的标签体系字符串转换成dict，以便往里增加新标签
feasLs = prdFeaStr.split('\n**')
feasDic = dict()
for f in feasLs:
    if f != '' and not f.isspace():
        k, substr = f.split('**')
        valls = substr.split('\n   - ')
        valset = set()
        for v in valls:
            if v != '' and not v.isspace():
                valset.add(v.strip())
        feasDic[k.strip()] = valset
feasDic

{'品牌': {'品牌知名度低', '品牌知名度高'},
 '价格': {'价格实惠', '价格昂贵', '性价比低', '性价比高'},
 '容量': {'容量过大', '容量过小', '容量适中'},
 '尺寸': {'便携性强', '占用空间小', '设计紧凑', '适合小家庭'},
 '设计': {'创新设计', '外观不美观', '外观美观', '设计传统'},
 '颜色': {'颜色不好看', '颜色不耐脏', '颜色好看', '颜色耐脏'},
 '重量': {'重量轻', '重量重'},
 '配件': {'配件缺失', '配件质量低', '配件质量高', '配件齐全'},
 '质量': {'产品易损', '产品耐用', '材质优良', '材质劣质'},
 '设备故障': {'设备故障频繁', '设备运行稳定'},
 '控制面板': {'操作复杂', '操作简单', '显示模糊', '显示清晰'},
 '照明': {'照明不足', '照明充足'},
 '定时器': {'定时不准', '定时准确'},
 '气味': {'无异味', '有异味'},
 '噪音': {'噪音大', '噪音小'},
 '物联网功能': {'连接不稳定', '连接稳定'},
 '能效': {'能耗低', '能耗高'},
 '清洁与维护': {'易于清洁', '难以清洁'},
 '涂层': {'涂层易脱落', '涂层耐用'},
 '安全性': {'安全性高', '安全隐患'},
 '老化': {'老化速度快', '老化速度慢'},
 '做工': {'做工粗糙', '做工精细'},
 '变色': {'不易变色', '易变色'},
 '材料': {'材料不环保', '材料环保'},
 '开关问题': {'开关失灵', '开关灵敏'},
 '按钮': {'按钮失灵', '按钮灵敏'},
 '旋钮': {'旋钮卡顿', '旋钮操作顺畅'},
 '烹饪功能': {'烹饪速度快', '烹饪速度慢'},
 '保温效果': {'保温持久', '保温效果差'},
 '蒸煮效果': {'蒸煮不均匀', '蒸煮均匀'},
 '触感': {'外壳不烫手', '外壳烫手'},
 '蒸笼': {'蒸笼容量大', '蒸笼容量小'},
 '内胆': {'内胆易损', '内胆耐用'},
 '盖子': {'盖子易开合'

In [23]:
import json
def converStr2Json(rspstr:str):
    """ 
    从评论中抽取出json字符串
    """
    jstr = rspstr.split('```json')[1].split('```')[0]
    jobj = json.loads(jstr)
    print(jobj)
    return jobj

In [24]:
from paddlenlp import Taskflow
similarity_task = Taskflow("text_similarity")

def chck_sim(wrd1, wrd2, thrshld=0.9):
    """ 
    计算两个中文的相似度，超过阈值则返回True
    """
    result = similarity_task([[wrd1, wrd2]])
    return result[0]['similarity'] > thrshld

def chck_sim_in_set(wrd1, wrdset, thrshld=0.9):
    """ 
    计算一个中文与一个集合中每个中文的相似度，若有一个超过阈值就返回True，
    """
    for w in wrdset:
        if chck_sim(wrd1, w, thrshld):
            print(f"find sim: >> {wrd1}  AND  {w}")
            return True
    return False

[2024-11-12 14:44:52,225] [    INFO] - We are using (<class 'paddlenlp.transformers.bert.tokenizer.BertTokenizer'>, False) to load 'simbert-base-chinese'.
[2024-11-12 14:44:52,237] [    INFO] - tokenizer config file saved in /home/luogang/.paddlenlp/models/simbert-base-chinese/tokenizer_config.json
[2024-11-12 14:44:52,238] [    INFO] - Special tokens file saved in /home/luogang/.paddlenlp/models/simbert-base-chinese/special_tokens_map.json


In [25]:
# 将生成评论中的标签中不存在于原标签体系的，插入原标签体系，形成新的更大的标签体系（产品特性标签）
print(feasDic)
thrshld = 0.7
genFeaDict = dict()
revLs = []
revLs.extend(converStr2Json(rspSynszRev0.choices[0].message.content.strip()))
revLs.extend(converStr2Json(rspSynszRev1.choices[0].message.content.strip()))
for jobj in revLs:
    feaLs = [f.strip() for f in jobj['Features'].split(',')]
    feaLs.extend( [f.strip() for f in jobj['NewFeatures'].split(',')] )
    print(feaLs)
    for fl in feaLs:
        if fl != '' and not fl.isspace() and fl not in feasDic and fl not in genFeaDict:
            if '/' not in fl or 'NewFeatures' in fl.split('/')[0]:
                fp = fl if '/' not in fl else fl.split('/')[1].strip()
                fs = None
            elif '/' in fl:
                fp, fs = fl.split('/')
            else:
                fp = None
                fs = None
                continue
            if fp not in feasDic:
                if not chck_sim_in_set(fp, feasDic.keys(), thrshld) and not chck_sim_in_set(fp, genFeaDict.keys(), thrshld):
                    print(f"-->add K {str(fp)}, V {str(fs)}")
                    genFeaDict[fp] = set()
                    if fs is not None:
                        genFeaDict[fp].add(fs)
            else:
                if fs and fs not in feasDic[fp] and not chck_sim_in_set(fs, feasDic[fp], thrshld):
                    print(f"**>add K {str(fp)}, V {str(fs)}")
                    feasDic[fp].add(fs)
                
print(genFeaDict)
for f in genFeaDict:
    feasDic[f] = genFeaDict[f]
print(feasDic)

/home/luogang/anaconda3/lib/python3.12/site-packages/paddlenlp/transformers/tokenizer_utils_base.py:1899: UserWarning: Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
  warnings.warn(


{'品牌': {'品牌知名度高', '品牌知名度低'}, '价格': {'性价比低', '价格实惠', '价格昂贵', '性价比高'}, '容量': {'容量过小', '容量适中', '容量过大'}, '尺寸': {'便携性强', '设计紧凑', '适合小家庭', '占用空间小'}, '设计': {'设计传统', '外观美观', '创新设计', '外观不美观'}, '颜色': {'颜色不好看', '颜色耐脏', '颜色不耐脏', '颜色好看'}, '重量': {'重量重', '重量轻'}, '配件': {'配件缺失', '配件质量高', '配件质量低', '配件齐全'}, '质量': {'材质优良', '产品耐用', '材质劣质', '产品易损'}, '设备故障': {'设备运行稳定', '设备故障频繁'}, '控制面板': {'操作简单', '显示清晰', '显示模糊', '操作复杂'}, '照明': {'照明充足', '照明不足'}, '定时器': {'定时准确', '定时不准'}, '气味': {'有异味', '无异味'}, '噪音': {'噪音大', '噪音小'}, '物联网功能': {'连接不稳定', '连接稳定'}, '能效': {'能耗低', '能耗高'}, '清洁与维护': {'易于清洁', '难以清洁'}, '涂层': {'涂层易脱落', '涂层耐用'}, '安全性': {'安全隐患', '安全性高'}, '老化': {'老化速度慢', '老化速度快'}, '做工': {'做工粗糙', '做工精细'}, '变色': {'易变色', '不易变色'}, '材料': {'材料环保', '材料不环保'}, '开关问题': {'开关失灵', '开关灵敏'}, '按钮': {'按钮灵敏', '按钮失灵'}, '旋钮': {'旋钮操作顺畅', '旋钮卡顿'}, '烹饪功能': {'烹饪速度快', '烹饪速度慢'}, '保温效果': {'保温效果差', '保温持久'}, '蒸煮效果': {'蒸煮不均匀', '蒸煮均匀'}, '触感': {'外壳不烫手', '外壳烫手'}, '蒸笼': {'蒸笼容量大', '蒸笼容量小'}, '内胆': {'内胆耐用', '内胆易损'}, '盖子': {'盖子难开合', '盖子易开合'}, '量杯': {'量杯准确', '量杯不准'

In [26]:
# 新的标签体系输出成文本形式，以便后续用于prompt中
feaLvlStr = ''
for k in feasDic.keys():
    feaLvlStr = feaLvlStr + '\n* ' + k
    if len(feasDic[k]):
        for s in feasDic[k]:
            feaLvlStr = feaLvlStr + '\n  - ' + s
print(feaLvlStr)


* 品牌
  - 品牌知名度高
  - 品牌知名度低
* 价格
  - 性价比低
  - 价格实惠
  - 价格昂贵
  - 性价比高
* 容量
  - 容量过小
  - 容量适中
  - 容量过大
* 尺寸
  - 便携性强
  - 设计紧凑
  - 适合小家庭
  - 占用空间小
* 设计
  - 设计传统
  - 外观美观
  - 创新设计
  - 外观不美观
* 颜色
  - 颜色不好看
  - 颜色耐脏
  - 颜色不耐脏
  - 颜色好看
* 重量
  - 重量重
  - 重量轻
* 配件
  - 配件缺失
  - 配件质量高
  - 配件质量低
  - 配件齐全
* 质量
  - 材质优良
  - 产品耐用
  - 材质劣质
  - 产品易损
* 设备故障
  - 设备运行稳定
  - 设备故障频繁
* 控制面板
  - 操作简单
  - 显示清晰
  - 显示模糊
  - 操作复杂
* 照明
  - 照明充足
  - 照明不足
* 定时器
  - 定时准确
  - 定时不准
* 气味
  - 有异味
  - 无异味
* 噪音
  - 噪音大
  - 噪音小
* 物联网功能
  - 连接不稳定
  - 连接稳定
* 能效
  - 能耗低
  - 能耗高
* 清洁与维护
  - 易于清洁
  - 难以清洁
* 涂层
  - 涂层易脱落
  - 涂层耐用
* 安全性
  - 安全隐患
  - 安全性高
* 老化
  - 老化速度慢
  - 老化速度快
* 做工
  - 做工粗糙
  - 做工精细
* 变色
  - 易变色
  - 不易变色
* 材料
  - 材料环保
  - 材料不环保
* 开关问题
  - 开关失灵
  - 开关灵敏
* 按钮
  - 按钮灵敏
  - 按钮失灵
* 旋钮
  - 旋钮操作顺畅
  - 旋钮卡顿
* 烹饪功能
  - 烹饪速度快
  - 烹饪速度慢
* 保温效果
  - 保温效果差
  - 保温持久
* 蒸煮效果
  - 蒸煮不均匀
  - 蒸煮均匀
* 触感
  - 外壳不烫手
  - 外壳烫手
* 蒸笼
  - 蒸笼容量大
  - 蒸笼容量小
* 内胆
  - 内胆耐用
  - 内胆易损
* 盖子
  - 盖子难开合
  - 盖子易开合
* 量杯
  - 量杯准确
  - 量杯不准
* 勺子
  - 勺子实用
  -

#### 抽取新的服务类标签

代码的作用和说明与上面‘抽取新的产品特性标签’类似。

In [27]:
revLs = []
revLs.extend(converStr2Json(rspSynszRev0.choices[0].message.content.strip()))
revLs.extend(converStr2Json(rspSynszRev1.choices[0].message.content.strip()))
for ujkey in srvTagDict:
    srvDic = srvTagDict[ujkey]
    print(srvDic)
    thrshld = 0.7
    genSrvDict = dict()
    for jobj in revLs:
        if jobj['UserJourneyStage'] != ujkey:
            continue
        feaLs = [f.strip() for f in jobj['Services'].split(',')]
        feaLs.extend( [f.strip() for f in jobj['NewServices'].split(',')] )
        print(feaLs)
        for fl in feaLs:
            if fl != '' and not fl.isspace() and fl not in srvDic and fl not in genSrvDict:
                if '/' not in fl or 'NewServices' in fl.split('/')[1]:
                    fp = fl if '/' not in fl else fl.split('/')[2].strip()
                    fs = None
                elif '/' in fl:
                    _, fp, fs = fl.split('/')
                else:
                    fp = None
                    fs = None
                    continue
                if fp not in srvDic:
                    if not chck_sim_in_set(fp, srvDic.keys(), thrshld) and not chck_sim_in_set(fp, genSrvDict.keys(), thrshld):
                        print(f"-->add K {str(fp)}, V {str(fs)}")
                        genSrvDict[fp] = set()
                        if fs is not None:
                            genSrvDict[fp].add(fs)
                else:
                    if fs and fs not in srvDic[fp] and not chck_sim_in_set(fs, srvDic[fp], thrshld):
                        print(f"**>add K {str(fp)}, V {str(fs)}")
                        srvDic[fp].add(fs)
    print(genSrvDict)
    for f in genSrvDict:
        srvDic[f] = genSrvDict[f]
    srvTagDict[ujkey] = srvDic
    print(srvDic)
print(srvTagDict)

[{'UserJourneyStage': 'Awareness and Research（意识和研究）', 'Sentiments': 'positive', 'Features': '品牌/品牌知名度高,设计/外观美观', 'NewFeatures': '功能数量/功能多', 'Services': '', 'NewServices': '', 'Review': "I've been researching rice cookers and found that the Zojirushi brand is highly recommended. Its sleek design and multiple cooking functions make it a top contender for my purchase."}, {'UserJourneyStage': 'Consideration and Purchase（考虑与购买）', 'Sentiments': 'positive', 'Features': '价格/性价比高,容量/容量适中', 'NewFeatures': '', 'Services': 'Consideration and Purchase（考虑与购买）/售前客服支持/信息准确性高', 'NewServices': '', 'Review': 'After comparing several models, I decided on this rice cooker because it offers great value for money and the perfect capacity for my family. The customer service was very helpful in providing accurate information.'}, {'UserJourneyStage': 'Delivery and Setup（交付和安装）', 'Sentiments': 'negative', 'Features': '配件/配件缺失', 'NewFeatures': '', 'Services': 'Delivery and Setup（交付和安装）/配送服务/包装破损', 'NewServices':

In [28]:
srvLvlStr = ''
for k in srvTagDict.keys():
    srvLvlStr = srvLvlStr + '\n**' + k + '**'
    if len(srvTagDict[k]):
        s = srvTagDict[k]
        for sk in s.keys():
            srvLvlStr = srvLvlStr + '\n  * ' + sk
            if len(s[sk]):
                for s2 in s[sk]:
                    srvLvlStr = srvLvlStr + '\n    - ' + s2
    srvLvlStr = srvLvlStr + '\n'
print(srvLvlStr)


**Awareness and Research（意识和研究）**

**Consideration and Purchase（考虑与购买）**
  * 售前客服支持
    - 信息不准确
    - 问题解决能力弱
    - 响应速度快
    - 服务态度好
    - 信息准确性高
    - 响应速度慢
    - 问题解决能力强
    - 服务态度差

**Delivery and Setup（交付和安装）**
  * 配送服务
    - 包装完整
    - 包装破损
    - 无运输损坏
    - 配送速度快
    - 配送速度慢
    - 运输损坏
  * 安装和使用指导
    - 语言易懂
    - 在线教程无用
    - 安装服务质量低
    - 安装服务质量高
    - 说明书清晰
    - 说明书不清晰
    - 自行安装困难
    - 语言难懂
    - 自行安装方便
    - 在线教程有用

**Ongoing Use and Maintenance（持续使用和维护）**
  * 使用指南和教程
    - 清洁和维护指导详细
    - 在线资源丰富
    - 清洁和维护指导不足
    - 功能说明不清晰
    - 在线资源匮乏
    - 功能说明清晰

**Customer Service（客户服务）**
  * 售后支持
    - 保修政策合理
    - 更换服务高效
    - 维修服务响应慢
    - 更换服务低效
    - 维修服务响应快
    - 保修政策不合理
  * 退货服务
    - 退货流程复杂
    - 退款速度慢
    - 退货流程简便
    - 退款速度快

**Feedback and Advocacy（反馈和倡导）**
  * 用户反馈收集
    - 反馈响应快
    - 反馈渠道单一
    - 反馈响应慢
    - 反馈渠道多样



## 基于真实评论打标的同时补充标签体系

与上面生成评论的同时打标与补充标签类似，只是这次换成了真实的评论，而上面生成的评论可作为few-shot使用。

In [29]:
history[0] = {"role": "system", "content": sysp_default} # 恢复资深专家身份

opin_prmp = """你是一个在电饭锅领域很资深的电商专家,擅长分析用户的评论,从评论中抽取用户的关键观点，也擅长总结标签体系，把标签体系中的标签打到用户的观点上，使标签与用户观点关联，又或者把观点抽象成标签体系中新的标签. 作为电饭锅领域的资深电商专家,你分析用户评论,从评论中抽取用户观点,并用简明的语言描述出来. 抽取的观点不一定只有一个.

# 参考的用户旅程阶段如下，在对评论进行分析时必须把评论归类到下面罗列的用户旅程阶段中的一个：

{ujstages}

# 参考的电饭锅服务关注点标签体系如下，服务关注点标签体系的一级标签是服务的所属用户旅程阶段（一般和生成的评论的所属用户旅程阶段一致），二级标签是服务类型，三级标签是用户在使用服务时可能对某系服务关注点的评论。你需要对从评论中抽取出来的每一条观点都打上服务相关的标签，以表示该观点表达了对哪些服务的哪些方面的满意或不满意的评价，但你打标签时可以不局限于该标签体系，可以对其进行增加。

{srvs}

# 参考的电饭锅特性标签体系如下，特性标签体系的一级标签是选择和评价电饭锅时索要考虑的点或方面、功能特性、部件、效果等，二级标签用户在选择或使用电饭锅后可能对某些产品功能特性做出的评价。你需要对从评论中抽取出来的每一条观点都打上电饭锅特性相关的标签，以表示这条观点涉及了哪些挑选和评价电饭锅时所需要关注的功能特性、部件等、但你打标签时可以不局限于该标签体系，可以对其进行增加。

{feas}


# 抽取观点的步骤如下:

### 一. 分析评论:

  1. **评论要表达的 sentiments**: 评论中的内容是 positive (也就是想表扬), 还是 negative (也就是想批评), 或者既有表扬也有批评. 如果既不是表扬也不是批评, 则属于 neutral .
  2. **电饭锅品牌 brand**: 评论中涉及的电饭锅的品牌. 若无法分析出评论涉及的品牌,则回答"不知道".
  3. **电饭锅型号 mode**: 评论中涉及的电饭锅的型号. 若无法分析出评论涉及的型号,则回答"不知道".
  4. **涉及的电饭锅属性 feature**: 评论中涉及到了电饭锅的什么属性, 什么功能特点, 什么部件. 比如: inner pot的尺寸大小, 煮蛋糕的功能, 等等. 若分析不出来则回答"不知道".
  5. **所属的用户旅程阶段**：该评论反映了用户正处于用户旅程的哪个阶段。注意，只能划分到一个用户旅程阶段中。

### 二. 抽取并罗列评论中的用户观点并输出为json格式

  1. 评论中的观点可能不止一个.每个观点应描述了这个电饭锅的特定属性, 功能特点, 或部件等是否好用, 或者在什么场景下是好用还是不好用的. 
  2. 把这个观点是从评论中的哪段文字总结得到的推演过程写出来. 比如: "make sure its on white rice function and hit start. that‘s it" -> "简单易用"
  3. 观点也需要区分 Sentiment 是 positive 还是 negative, 如果这条观点既不是表扬也不是批评, 则 Sentiment 为 neutral . 此时该观点认为是"无具体观点", 可以不罗列出来.
  4. 从评论中抽取出来的每一条观点都打上电饭锅特性相关的标签，以表示这条观点涉及了哪些挑选和评价电饭锅时所需要关注的功能特性、部件等。打上的特性标签以“父特性/子特性”的格式填在"Features"中，多个特性用逗号分隔。"Features"中的标签必须来自上述用于参考的电饭锅特性标签体系。
  5. 从评论中抽取出来的每一条观点都打上服务相关的标签，以表示该观点表达了对哪些服务的哪些方面的满意或不满意的评价。打上的服务标签以“所属用户旅程阶段/父服务类型/子服务关注点”的格式填在"Services"中，多个特性用逗号分隔。"Services"中的标签必须来自上述用于参考的电饭锅服务关注点标签体系。
  6. 显示抽取出来的观点 Opinion, 若无具体观点, 可以显示空字符串.
  7. 将抽取出来的观点重写（Paraphrase）成更通用的，能简明反映观点的短语。
  8. 如果现有的电饭锅特性标签体系中的标签无法表达该观点中涉及的电饭锅的功能特性，则在"NewFeatures"中以“父特性/子特性”的格式填入该观点所涉及的原标签体系中没有的功能特性。
  9. 如果现有的电饭锅服务关注点标签体系中的标签无法表达该观点中涉及的电饭锅相关服务的某些能引起用户满意或不满的关注点，则在"NewServices"中以“所属用户旅程阶段/父服务类型/子服务关注点”的格式填入该观点所涉及的原标签体系中没有的服务关注点。


# 举例如下:

## 例子一:

请分析下述评论并抽取观点:'I want to buy a rice cooker.  And I asked my friend for suggestion.  He says that the best features of his Tiger rice cooker are the detachable inner pot and the quick-cook function. For him as a single person, the detachable inner pot makes it easy to clean the rice cooker, and the quick-cook function shortens his cooking time.'

### 一. 分析评论
  1. **评论要表达的 sentiments**: positive .
  2. **电饭锅品牌 brand**: Tiger.
  3. **电饭锅型号 mode**: 不知道.
  4. **涉及的电饭锅属性 features**: 快煮功能，可拆卸内胆.
  5. **所属的用户旅程阶段**：Awareness and Research（意识和研究）

### 二. 抽取并罗列评论中的用户观点并输出为json格式

```json
{{
  "Opinions" : [
    {{
      "Reasonging": "For him as a single person, the detachable inner pot makes it easy to clean the rice cooker -> 可拆卸内胆使得电饭锅易清洁.",
      "Sentiments":"positive",
      "Features":"清洁与维护/易于清洁",
      "Services": "",
      "Opinion": "可拆卸内胆使得电饭锅易清洁",
      "Paraphrase": "可拆卸内胆使得清洁容易",
      "NewFeatures":"内胆/内胆易拆卸",
      "NewServices": ""
    }},
    {{
      "Reasonging": "and the quick-cook function shortens his cooking time -> 煮饭快省时间.",
      "Sentiments":"positive",
      "Features":"烹饪功能/烹饪速度快",
      "Services": "",
      "Opinion": "煮饭快省时间",
      "Paraphrase": "煮饭快省时间",
      "NewFeatures":"",
      "NewServices": ""
    }}
  ]
}}
```

## 例子二:

请分析下述评论并抽取观点:'Effortlessly perfect rice every time, Zojirushi is a game changer!'

### 一. 分析评论
  1. **评论要表达的 sentiments**: positive .
  2. **电饭锅品牌 brand**: Zojirushi.
  3. **电饭锅型号 mode**: 不知道.
  4. **涉及的电饭锅属性 features**: 不知道.
  5. **所属的用户旅程阶段**：不知道

### 二. 抽取并罗列评论中的用户观点并输出为json格式

```json
{{
  "Opinions" : [
    {{
      "Reasonging": "不涉及具体的电饭锅的属性,功能特点,部件等 -> 无具体观点",
      "Sentiments":"positive",
      "Features":"",
      "Services": "",
      "Opinion": "无具体观点",
      "Paraphrase": "",
      "NewFeatures":"",
      "NewServices": ""
    }}
  ]
}}
```

## 例子三:

请分析下述评论并抽取观点:'I received Zojirushi NS-TSC10 rice cooker a week ago. This is my first rice cooker. So far I have used it twice: cooked white rice and baked a cheese cake. The rice does not squash together as regular pot would do. The texture is a slightly different between rice cooker and regular pot. I like the taste better from this rice cooker.About cheese cake, the cake diameter is about 6 to 7 inches. It may be suitable for visiting friend or small group. it may not be big enough for multi family party.The only trivial con about this product: I wish the lid could be opened a little more. Every time I took the container out or put it back in, it would hit the lid. It is just a little touch. It will not cause any issue or damage, just a little annoying.'

### 一. 分析评论
  1. **评论要表达的 sentiments**: positive, negative .
  2. **电饭锅品牌 brand**: Zojirushi.
  3. **电饭锅型号 mode**: NS-TSC10.
  4. **涉及的电饭锅属性 features**: 做饭, 做蛋糕, 容量, 可拆卸性
  5. **所属的用户旅程阶段**：Feedback and Advocacy（反馈和倡导）

### 二. 抽取并罗列评论中的用户观点并输出为json格式

```json
{{
  "Opinions" : [
    {{
      "Reasonging": "The rice does not squash together as regular pot would do -> 做饭功能做出来的米饭不会粘在一起.",
      "Sentiments":"positive",
      "Features":"",
      "Services": "",
      "Opinion": "煮饭功能做出来的米饭不会粘在一起",
      "Paraphrase": "米饭不粘一起",
      "NewFeatures":"煮饭/米饭不粘一起",
      "NewServices": ""
    }},
    {{
      "Reasonging": "The texture is a slightly different between rice cooker and regular pot -> 不涉及表扬或批评 -> 无具体观点.",
      "Sentiments":"neutral",
      "Features":"",
      "Services": "",
      "Opinion": "无具体观点",
      "Paraphrase": "",
      "NewFeatures":"",
      "NewServices": ""
    }},
    {{
      "Reasonging": "I like the taste better from this rice cooker -> 做饭功能做出来的米饭口味更好.",
      "Sentiments":"positive",
      "Features":"",
      "Services": "",
      "Opinion": "做饭功能做出来的米饭口味更好",
      "Paraphrase": "米饭口味好",
      "NewFeatures":"煮饭/米饭口味好",
      "NewServices": ""
    }},
    {{
      "Reasonging": "the cake diameter is about 6 to 7 inches. It may be suitable for visiting friend or small group -> 这种容量做出来的蛋糕适合小型聚会.",
      "Sentiments":"positive",
      "Features":"",
      "Services": "",
      "Opinion": "这种容量做出来的蛋糕适合小型聚会",
      "Paraphrase": "容量适合少人",
      "NewFeatures":"容量/容量适合少人",
      "NewServices": ""
    }},
    {{
      "Reasonging": "the cake diameter is about 6 to 7 inches. It may be suitable for visiting friend or small group. it may not be big enough for multi family party" -> 这种容量做出来的蛋糕对于大型聚会来说不够大.",
      "Sentiments":"negative",
      "Features":"",
      "Services": "",
      "Opinion": "这种容量做出来的蛋糕对于大型聚会来说不够大",
      "Paraphrase": "容量不适合多人",
      "NewFeatures":"容量/容量不适合多人",
      "NewServices": ""
    }},
    {{
      "Reasonging": "The only trivial con about this product: I wish the lid could be opened a little more. Every time I took the container out or put it back in, it would hit the lid -> 内部容器较难拆卸,会被盖子挡住.",
      "Sentiments":"negative",
      "Features":"",
      "Services": "",
      "Opinion": "内部容器较难拆卸,会被盖子挡住",
      "Paraphrase": "容器拆卸难",
      "NewFeatures":"容器/容器拆卸难",
      "NewServices": ""
    }},
  ]
}}
```

## 例子四

请分析下述评论并抽取观点:'I love the color of this rice cooker, and the customer service was very helpful in answering my questions.'

### 一. 分析评论
  1. **评论要表达的 sentiments**: positive .
  2. **电饭锅品牌 brand**: 不知道.
  3. **电饭锅型号 mode**: 不知道.
  4. **涉及的电饭锅属性 features**: 颜色
  5. **所属的用户旅程阶段**：Consideration and Purchase（考虑与购买）

### 二. 抽取并罗列评论中的用户观点并输出为json格式

```json
{{
  "Opinions" : [
    {{
      "Reasonging": "I love the color of this rice cooker -> 颜色好看.",
      "Sentiments":"positive",
      "Features":"颜色/颜色好看",
      "Services": "",
      "Opinion": "颜色好看",
      "Paraphrase": "颜色好看",
      "NewFeatures":"",
      "NewServices": ""
    }},
    {{
      "Reasonging": "and the customer service was very helpful in answering my questions -> 售前客服支持有帮助.",
      "Sentiments":"positive",
      "Features":"",
      "Services": "Consideration and Purchase（考虑与购买）/售前客服支持/问题解决能力强",
      "Opinion": "售前客服支持有帮助",
      "Paraphrase": "售前客服支持有用",
      "NewFeatures":"",
      "NewServices": ""
    }}
  ]
}}

## 例子五

请分析下述评论并抽取观点:'Effortlessly perfect rice every time, Zojirushi is a game changer!'

### 一. 分析评论
1. **评论要表达的 sentiments**: positive.
2. **电饭锅品牌 brand**: Zojirushi.
3. **电饭锅型号 mode**: 不知道.
4. **涉及的电饭锅属性 features**: 不知道.
5. **所属的用户旅程阶段**：Feedback and Advocacy（反馈和倡导）

### 二. 抽取并罗列评论中的用户观点并输出为json格式

```json
{{
  "Opinions" : [
    {{
      "Reasonging": "Effortlessly perfect rice every time -> 煮饭效果好",
      "Sentiments":"positive",
      "Features":"煮饭/煮饭效果好",
      "Services": "",
      "Opinion": "煮饭效果好",
      "Paraphrase": "",
      "NewFeatures":"",
      "NewServices": ""
    }},
    {{
      "Reasonging": "Zojirushi is a game changer! -> 品牌好",
      "Sentiments":"positive",
      "Features":"品牌/品牌知名度高",
      "Services": "",
      "Opinion": "品牌好",
      "Paraphrase": "",
      "NewFeatures":"",
      "NewServices": ""
    }}
  ]
}}
```

## 举例结束

请分析下述评论并抽取观点:'{review}'"""



In [30]:
# revprmp = opin_prmp.format(ujstages= ujStageStr, srvs= srvLvlStr, feas= feaLvlStr,
#                           # review="I bought this cooker to replace one I had for many years so I did an upgrade, although my other cooler still worked fine. This one works like a charm, glad I bought it.")
#                           # review="I‘ve been making a crazy amount of different rice recipes for months.  I love this thing.  Best food prep investment I have made.")
#                           # review="Effortlessly perfect rice every time, Zojirushi is a game changer!")
#                           review="weejio review coming soon.")
# rev_rsp = ask_with_history(revprmp)
# revobjs = converStr2Json(rev_rsp.choices[0].message.content.strip())

### 从GPT输出中抽取标签并写到excel

In [31]:
# 从GPT的output中抽取出产品特性标签并补充标签体系
def extract_fea(revobjs, feasDic):
    print(feasDic)
    thrshld = 0.7
    genFeaDict = dict()
    tagls = []
    for jobj in revobjs['Opinions']:
        feaLs = [f.strip() for f in jobj['Features'].split(',')]
        feaLs.extend( [f.strip() for f in jobj['NewFeatures'].split(',')] )
        print(feaLs)
        for fl in feaLs:
            if fl != '' and not fl.isspace() and fl not in feasDic and fl not in genFeaDict:
                if '/' not in fl or 'NewFeatures' in fl.split('/')[0]:
                    fp = fl if '/' not in fl else fl.split('/')[1].strip()
                    fs = None
                    tagls.append( (fp, fp) )
                elif '/' in fl:
                    fp, fs = fl.split('/')
                    tagls.append( (fp, fs) )
                else:
                    fp = None
                    fs = None
                    continue
                if fp not in feasDic:
                    if not chck_sim_in_set(fp, feasDic.keys(), thrshld) and not chck_sim_in_set(fp, genFeaDict.keys(), thrshld):
                        print(f"-->add K {str(fp)}, V {str(fs)}")
                        genFeaDict[fp] = set()
                        if fs is not None:
                            genFeaDict[fp].add(fs)
                else:
                    if fs and fs not in feasDic[fp] and not chck_sim_in_set(fs, feasDic[fp], thrshld):
                        print(f"**>add K {str(fp)}, V {str(fs)}")
                        feasDic[fp].add(fs)
                    
    print(genFeaDict)
    for f in genFeaDict:
        feasDic[f] = genFeaDict[f]
    print(feasDic)
    return feasDic, tagls

In [32]:
# 从GPT的output中抽取出服务标签并补充标签体系
def extract_srv(ujname, revobjs, srvTagDict):
    if ujname not in srvTagDict:
        return srvTagDict, []
    srvDic = srvTagDict[ujname]
    print(srvDic)
    thrshld = 0.7
    genSrvDict = dict()
    tagls = []
    for jobj in revobjs['Opinions']:
        # if ujname != ujkey:
        #     continue
        feaLs = [f.strip() for f in jobj['Services'].split(',')]
        feaLs.extend( [f.strip() for f in jobj['NewServices'].split(',')] )
        print(feaLs)
        for fl in feaLs:
            if fl != '' and not fl.isspace() and fl not in srvDic and fl not in genSrvDict:
                if '/' not in fl or 'NewServices' in fl.split('/')[1]:
                    fp = fl if '/' not in fl else fl.split('/')[2].strip()
                    fs = None
                    tagls.append( (ujname, fp, fp) )
                elif '/' in fl:
                    _, fp, fs = fl.split('/')
                    tagls.append( (ujname, fp, fs) )
                else:
                    fp = None
                    fs = None
                    continue
                if fp not in srvDic:
                    if not chck_sim_in_set(fp, srvDic.keys(), thrshld) and not chck_sim_in_set(fp, genSrvDict.keys(), thrshld):
                        print(f"-->add K {str(fp)}, V {str(fs)}")
                        genSrvDict[fp] = set()
                        if fs is not None:
                            genSrvDict[fp].add(fs)
                else:
                    if fs and fs not in srvDic[fp] and not chck_sim_in_set(fs, srvDic[fp], thrshld):
                        print(f"**>add K {str(fp)}, V {str(fs)}")
                        srvDic[fp].add(fs)
    print(genSrvDict)
    for f in genSrvDict:
        srvDic[f] = genSrvDict[f]
    srvTagDict[ujname] = srvDic
    print(srvDic)
    print(srvTagDict)
    return srvTagDict, tagls


In [33]:
import pandas as pd

def tag_and_extract(rev, processed_data):
    global feasDic, srvTagDict
    revprmp = opin_prmp.format(ujstages= ujStageStr, srvs= srvLvlStr, feas= feaLvlStr,
                           review=rev)
    rev_rsp = ask_with_history(revprmp)
    # 从GPT的输出中抽取json
    revobjs = converStr2Json(rev_rsp.choices[0].message.content.strip())
    # 抽取用户旅程阶段
    ujname = rev_rsp.choices[0].message.content.strip().split('### 二. 抽取并罗列评论中的用户观点并输出为json格式')[0].strip().split('**所属的用户旅程阶段**：')[1].strip().split(',')[0]
    # 抽取特征标签
    feasDic, feaTagLs = extract_fea(revobjs, feasDic)
    # 抽取服务标签
    srvTagDict, srvTagLs = extract_srv(ujname, revobjs, srvTagDict)
    # 打印excel
    for tup in feaTagLs:
        newrow = pd.DataFrame({'Original': rev, 'LABEL_NAME': f'{ujname}/Usage Issues（产品相关问题）',    'LABEL_SECONDARY_NAME': tup[0], 'LABEL_THIRD_NAME':tup[1]},  index=[0])
        processed_data = pd.concat([processed_data, newrow], ignore_index=True)  
    for tup in srvTagLs:
        newrow = pd.DataFrame({'Original': rev, 'LABEL_NAME': tup[0],    'LABEL_SECONDARY_NAME': tup[1], 'LABEL_THIRD_NAME':tup[2]},  index=[0])
        processed_data = pd.concat([processed_data, newrow], ignore_index=True)
    if len(feaTagLs) == 0 and len(srvTagLs) == 0:
        ujname = 'Others' if '不知道' in ujname else ujname
        newrow = pd.DataFrame({'Original': rev, 'LABEL_NAME': ujname,    'LABEL_SECONDARY_NAME': 'Others', 'LABEL_THIRD_NAME': 'Others'},  index=[0])
        processed_data = pd.concat([processed_data, newrow], ignore_index=True)
    return processed_data

In [34]:

# processed_data0 = pd.DataFrame(columns=['Original', 'LABEL_NAME', 'LABEL_SECONDARY_NAME', 'LABEL_THIRD_NAME'])  
# rows = tag_and_extract("When we upgraded to this rice cooker it was like a breath of fresh air! We constantly have a pot of rice going and our old one was not cutting it. This makes rice perfectly every single time and never burns the rice to the pot. It’s very easy to clean and small enough to store away when not in use.", processed_data0)
# print(rows)

In [ ]:
import pandas as pd  
  
def process_excel(input_file, output_file, sheet_name, column_name):  
    """ 
    输入：包含真实评论的excel，以及真实评论所处的sheet和column
    输出：打标后的excel
    """
    global history
    # 读取Excel文件的指定sheet  
    df = pd.read_excel(input_file, sheet_name=sheet_name)  
  
    # 创建一个新的DataFrame来存储处理后的数据，列名分别为真实评论‘Original’和各级标签
    processed_data = pd.DataFrame(columns=['Original', 'LABEL_NAME', 'LABEL_SECONDARY_NAME', 'LABEL_THIRD_NAME'])  
  
    # 遍历指定列中的每一行  
    for index, value in df[column_name].items():  
        # 确保是字符串类型  
        if isinstance(value, str):  
            print(f'>>> {value}')
            processed_data = tag_and_extract(value, processed_data)
            history = history[:-2]
  
    # 保存到新的Excel文件  
    processed_data.to_excel(output_file, index=False)  
  
# 使用示例  
input_excel = '/mnt/d/OneDrive - Microsoft/OneDOC/Accounts/美的/样例问题.xlsx'  
sheet_name = 'review'       # 要读取的Sheet名  
column_name = 'Original'     # 指定的列名  
output_excel = '/mnt/d/OneDrive - Microsoft/OneDOC/Accounts/美的/output3.xlsx'  
  
process_excel(input_excel, output_excel, sheet_name, column_name)  

>>> I use this product to make rice. The rice is better than the other cheaper rice cookers. I think it‘s worth the price.
### 一. 分析评论
1. **评论要表达的 sentiments**: positive.
2. **电饭锅品牌 brand**: 不知道.
3. **电饭锅型号 mode**: 不知道.
4. **涉及的电饭锅属性 features**: 煮饭, 价格.
5. **所属的用户旅程阶段**：Feedback and Advocacy（反馈和倡导）

### 二. 抽取并罗列评论中的用户观点并输出为json格式

```json
{
  "Opinions" : [
    {
      "Reasonging": "The rice is better than the other cheaper rice cookers -> 煮饭效果比其他便宜的电饭锅好",
      "Sentiments":"positive",
      "Features":"煮饭/煮饭效果好",
      "Services": "",
      "Opinion": "煮饭效果比其他便宜的电饭锅好",
      "Paraphrase": "煮饭效果好",
      "NewFeatures":"",
      "NewServices": ""
    },
    {
      "Reasonging": "I think it‘s worth the price -> 价格值得",
      "Sentiments":"positive",
      "Features":"价格/性价比高",
      "Services": "",
      "Opinion": "价格值得",
      "Paraphrase": "性价比高",
      "NewFeatures":"",
      "NewServices": ""
    }
  ]
}
```
{'Opinions': [{'Reasonging': 'The rice is better than the other cheaper r

### 显示补充标签后的产品特性标签体系

In [37]:
# 显示补充新标签后的产品特征标签体系
feaLvlStr2 = ''
for k in feasDic.keys():
    feaLvlStr2 = feaLvlStr2 + '\n* ' + k
    if len(feasDic[k]):
        for s in feasDic[k]:
            feaLvlStr2 = feaLvlStr2 + '\n  - ' + s
print(feaLvlStr2)


* 品牌
  - 品牌知名度高
  - 品牌知名度低
* 价格
  - 性价比低
  - 价格实惠
  - 价格昂贵
  - 性价比高
* 容量
  - 容量过小
  - 容量适中
  - 容量过大
* 尺寸
  - 便携性强
  - 设计紧凑
  - 适合小家庭
  - 占用空间小
* 设计
  - 设计传统
  - 外观美观
  - 创新设计
  - 外观不美观
* 颜色
  - 颜色不好看
  - 颜色耐脏
  - 颜色不耐脏
  - 颜色好看
* 重量
  - 重量重
  - 重量轻
* 配件
  - 配件缺失
  - 备件供应不足
  - 配件质量高
  - 配件齐全
  - 配件质量低
* 质量
  - 材质优良
  - 产品耐用
  - 材质劣质
  - 产品易损
* 设备故障
  - 设备运行稳定
  - 设备故障频繁
* 控制面板
  - 操作简单
  - 显示清晰
  - 显示模糊
  - 操作复杂
* 照明
  - 照明充足
  - 照明不足
* 定时器
  - 定时准确
  - 定时不准
* 气味
  - 有异味
  - 无异味
* 噪音
  - 噪音大
  - 噪音小
* 物联网功能
  - 连接不稳定
  - 连接稳定
* 能效
  - 能耗低
  - 能耗高
* 清洁与维护
  - 易于清洁
  - 难以清洁
* 涂层
  - 涂层易脱落
  - 涂层耐用
* 安全性
  - 安全隐患
  - 安全性高
* 老化
  - 老化速度慢
  - 老化速度快
* 做工
  - 做工粗糙
  - 做工精细
* 变色
  - 易变色
  - 不易变色
* 材料
  - 材料环保
  - 材料不环保
* 开关问题
  - 开关失灵
  - 开关灵敏
* 按钮
  - 按钮灵敏
  - 按钮失灵
* 旋钮
  - 旋钮操作顺畅
  - 旋钮卡顿
* 烹饪功能
  - 烹饪速度快
  - 烹饪速度慢
* 保温效果
  - 保温效果差
  - 保温持久
* 蒸煮效果
  - 蒸煮不均匀
  - 蒸煮均匀
* 触感
  - 外壳不烫手
  - 外壳烫手
* 蒸笼
  - 蒸笼容量大
  - 蒸笼容量小
* 内胆
  - 内胆耐用
  - 内胆易损
* 盖子
  - 盖子难开合
  - 盖子易开合
* 量杯
  - 量杯准确
  - 量杯不准
* 勺子
 

In [38]:
# 补充新标签后的产品特征标签体系
finalFeaStr = """ 

* 品牌
  - 品牌知名度低
  - 推荐
  - 品牌知名度高
* 价格
  - 性价比高
  - 价格昂贵
  - 性价比低
  - 价格实惠
* 容量
  - 容量适中
  - 容量过大
  - 容量过小
* 尺寸
  - 设计紧凑
  - 适合小家庭
  - 便携性强
  - 占用空间小
* 设计
  - 设计传统
  - 设计紧凑
  - 创新设计
  - 外观美观
  - 外观不美观
* 颜色
  - 颜色不耐脏
  - 颜色好看
  - 颜色耐脏
  - 颜色不好看
* 重量
  - 重量轻
  - 重量重
* 配件
  - 配件质量低
  - 配件质量高
  - 配件齐全
  - 配件缺失
* 质量
  - 产品耐用
  - 材质劣质
  - 材质优良
  - 产品易损
* 设备故障
  - 设备运行稳定
  - 设备故障频繁
* 控制面板
  - 显示模糊
  - 操作简单
  - 显示清晰
  - 操作复杂
* 照明
  - 照明充足
  - 照明不足
* 定时器
  - 定时不准
  - 定时准确
* 气味
  - 无异味
  - 有异味
* 噪音
  - 噪音小
  - 噪音大
* 物联网功能
  - 连接不稳定
  - 连接稳定
* 能效
  - 能耗高
  - 能耗低
* 清洁与维护
  - 易于清洁
  - 难以清洁
* 涂层
  - 涂层易脱落
  - 涂层耐用
* 安全性
  - 安全隐患
  - 安全性高
* 老化
  - 老化速度慢
  - 老化速度快
* 做工
  - 做工精细
  - 做工粗糙
* 变色
  - 易变色
  - 不易变色
* 材料
  - 材料环保
  - 材料不环保
* 开关问题
  - 开关灵敏
  - 开关失灵
* 按钮
  - 按钮灵敏
  - 按钮失灵
* 旋钮
  - 旋钮操作顺畅
  - 旋钮卡顿
* 烹饪功能
  - 烹饪速度慢
  - 烹饪速度快
  - 烹饪精准
* 保温效果
  - 保温持久
  - 保温效果差
* 蒸煮效果
  - 蒸煮均匀
  - 蒸煮不均匀
* 触感
  - 外壳不烫手
  - 外壳烫手
* 蒸笼
  - 蒸笼容量大
  - 蒸笼容量小
* 内胆
  - 内胆耐用
  - 内胆易损
* 盖子
  - 盖子难开合
  - 盖子易开合
* 量杯
  - 量杯不准
  - 量杯准确
* 勺子
  - 勺子不实用
  - 勺子实用
* 蒸汽阀
  - 蒸汽阀安全
  - 蒸汽阀不安全
* 橡胶垫
  - 橡胶垫易损
  - 橡胶垫耐用
* 密封圈
  - 密封圈密封性好
  - 密封圈漏气
* 水箱
  - 水箱容量大
  - 水箱容量小
* 铭牌
  - 铭牌模糊
  - 铭牌清晰
* 螺丝
  - 螺丝生锈
  - 螺丝不生锈
* 手柄
  - 手柄不舒适
  - 手柄舒适
* 生锈
  - 不易生锈
  - 易生锈
* 味道
  - 米饭无香味
  - 米饭香味浓
* 二维码
  - 二维码易扫描
  - 二维码难扫描
* 煮汤
  - 煮汤效果好
  - 煮汤效果差
* 煮饭
  - 米饭口感好
  - 简单易用
  - 煮饭效果差
  - 米饭不粘一起
  - 米饭易变味变黄
  - 煮饭效果好
  - 米饭不会烧焦
  - 不糊锅
  - 米饭口感差
* 煮粥
  - 煮粥效果好
  - 煮粥效果差
* 漏水
  - 漏水
  - 不漏水
* 电源线
  - 电源线长度合适
  - 电源线长度不合适
* 存储
  - 存储不便
  - 存储方便
* 功能数量
  - 功能有限
  - 功能多
* 易用性
  - 易于使用
* 声音提示
  - 声音提示可爱
  - 有提示音
* 使用
  - 易于使用
* 性能
  - 表现出色
* 用户群体
  - 适合西班牙裔家庭
"""

### 显示补充标签后的服务标签体系

In [39]:
# 显示补充新标签后的服务标签体系
srvLvlStr2 = ''
for k in srvTagDict.keys():
    srvLvlStr2 = srvLvlStr2 + '\n**' + k + '**'
    if len(srvTagDict[k]):
        s = srvTagDict[k]
        for sk in s.keys():
            srvLvlStr2 = srvLvlStr2 + '\n  * ' + sk
            if len(s[sk]):
                for s2 in s[sk]:
                    srvLvlStr2 = srvLvlStr2 + '\n    - ' + s2
    srvLvlStr2 = srvLvlStr2 + '\n'
print(srvLvlStr2)


**Awareness and Research（意识和研究）**

**Consideration and Purchase（考虑与购买）**
  * 售前客服支持
    - 信息不准确
    - 问题解决能力弱
    - 响应速度快
    - 服务态度好
    - 信息准确性高
    - 响应速度慢
    - 问题解决能力强
    - 服务态度差

**Delivery and Setup（交付和安装）**
  * 配送服务
    - 包装完整
    - 包装破损
    - 无运输损坏
    - 配送速度快
    - 配送速度慢
    - 运输损坏
  * 安装和使用指导
    - 语言易懂
    - 在线教程无用
    - 安装服务质量低
    - 安装服务质量高
    - 说明书清晰
    - 说明书不清晰
    - 自行安装困难
    - 语言难懂
    - 自行安装方便
    - 在线教程有用

**Ongoing Use and Maintenance（持续使用和维护）**
  * 使用指南和教程
    - 清洁和维护指导详细
    - 在线资源丰富
    - 清洁和维护指导不足
    - 功能说明不清晰
    - 在线资源匮乏
    - 功能说明清晰
  * 售后支持
    - 备件缺货

**Customer Service（客户服务）**
  * 售后支持
    - 保修政策合理
    - 更换服务高效
    - 维修服务响应慢
    - 更换服务低效
    - 维修服务响应快
    - 保修政策不合理
  * 退货服务
    - 退货流程复杂
    - 退款速度慢
    - 退货流程简便
    - 退款速度快

**Feedback and Advocacy（反馈和倡导）**
  * 用户反馈收集
    - 反馈响应快
    - 反馈渠道单一
    - 反馈响应慢
    - 反馈渠道多样



In [40]:
# 补充新标签后的服务标签体系
srvLvlStr2 = """ 
**Awareness and Research（意识和研究）**

**Consideration and Purchase（考虑与购买）**
  * 售前客服支持
    - 服务态度差
    - 响应速度慢
    - 响应速度快
    - 信息不准确
    - 服务态度好
    - 问题解决能力弱
    - 问题解决能力强
    - 信息准确性高

**Delivery and Setup（交付和安装）**
  * 配送服务
    - 包装完整
    - 包装破损
    - 运输损坏
    - 配件缺失
    - 配送速度慢
    - 无运输损坏
    - 配送速度快
  * 安装和使用指导
    - 说明书清晰
    - 语言易懂
    - 自行安装方便
    - 语言难懂
    - 在线教程无用
    - 安装服务质量高
    - 在线教程有用
    - 安装服务质量低
    - 自行安装困难
    - 说明书不清晰

**Ongoing Use and Maintenance（持续使用和维护）**
  * 使用指南和教程
    - 功能说明不清晰
    - 功能说明清晰
    - 在线资源丰富
    - 清洁和维护指导不足
    - 清洁和维护指导详细
    - 在线资源匮乏
  * 售后支持
    - 更换服务低效
    - 备件缺货
  * 备件供应
    - 备件缺失

**Customer Service（客户服务）**
  * 售后支持
    - 保修政策合理
    - 维修服务响应快
    - 更换服务低效
    - 保修政策不合理
    - 维修服务响应慢
    - 更换服务高效
  * 退货服务
    - 退款速度快
    - 退款速度慢
    - 退货流程复杂
    - 退货流程简便

**Feedback and Advocacy（反馈和倡导）**
  * 用户反馈收集
    - 反馈响应慢
    - 反馈响应快
    - 反馈渠道单一
    - 反馈渠道多样
"""

### 其它

若每次分析和打标一个评论较慢，可一次分析多个评论，并且调用batch接口。